In [1]:
import re #18/02/2023 works!
import pandas as pd
import numpy as np
import pdfplumber
import shutil
import os, glob

dir_remov = ["C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Json/",
             "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split/", 
             "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split/Transporeon_split_1",
             "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/", 
             "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/Chawan_Vopak_email/",
             "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Send_Vopak/"]
for dir_remo in dir_remov:
    for file_remov in glob.glob(dir_remo +"*.pdf"):
        os.remove(file_remov)

from pathlib import Path #remove file from dir
dir_path = Path('C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/')
pdf_files = dir_path.glob('*.pdf') # returns a list, get all pdf files in directory
# check if specific name exists in list
for pdf_file in pdf_files:
    if 'Transports_Number_50.pdf' in pdf_file.name:
        os.remove(pdf_file)
        
#move file, overwrite if exists
list_of_files = glob.glob('C:/Users/john.tan/Downloads/*.pdf') # *.csv
latest_file = max(list_of_files, key=os.path.getctime) #get latest files
src_path = r"C:/Users/john.tan/Downloads/"
dest_path = r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/"

def substring_after(s, delim):
    return s.partition(delim)[2]

latest_transport = substring_after(latest_file, "\\")
if os.path.isfile(os.path.join(dest_path, latest_transport)):
    os.remove(os.path.join(dest_path, latest_transport)) #remove file if exists
shutil.move(src_path+latest_transport, dest_path)


from PyPDF2 import PdfWriter, PdfReader #split pdf files
from PyPDF2 import PdfWriter, PdfReader
path_file = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transports_Number_50.pdf"
file_out = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split/Transport"
inputpdf = PdfReader(open(path_file, "rb"))

for i in range(len(inputpdf.pages)):
    output = PdfWriter()
    output.add_page(inputpdf.pages[i])
    with open(file_out + "_%s.pdf" % i, "wb") as outputStream:
        output.write(outputStream)
        
substr = 'Transporeon-ID' #remove pdf w/o 'Transporeon-ID'
path_dir = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split/"
for file in glob.glob(path_dir+"*.pdf"):
    file = str(file)
    pdf = pdfplumber.open(file)
    page = pdf.pages[0]
    text = page.extract_text()
    pdf.close()
    
    if text.find(substr) !=-1:
        path_from = file
        #print(file)
        path_dest = 'C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split_1/'
        shutil.move(path_from, path_dest)
        pdf.close()

        
regexp = re.compile("Origin(.*)$", re.MULTILINE)
path_dir_1 = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split_1/"
path_in = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/" #move pdf to 'Chawan', 'Vopak'
for file in glob.glob(path_dir_1 +"*.pdf"):
    file = str(file)
    path_from = file
    pdf = pdfplumber.open(file)
    page = pdf.pages[0]
    text = page.extract_text()
    pdf.close()
    try:
        ori = regexp.search(text).group(1).split('SG-',1)[0].split()[1]
        if ori in ['CHAWAN', 'VOPAK']:
            shutil.move(path_from, path_in)
            pdf.close()
    except:
        pass

In [2]:
dir = r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/" #regex rename pdf Chia MEAN
regexp = re.compile(r'(?s)Order type (.*?)\n[^\S\r\n]*Shipping ', re.MULTILINE)
regexp_2 = re.compile(r'(?s)Shipping number (BOL) (.*?)\n[^\S\r\n]*Gross ', re.MULTILINE)
regexp_3 = re.compile(r'Shipping number (BOL) (\d+)', re.MULTILINE)

for file in os.listdir(dir):
    f = os.path.join(dir, file) # iterate over files in dir
    if os.path.isfile(f): # checking if it is a file
        pdf = pdfplumber.open(f)
        page = pdf.pages[0]
        text = page.extract_text()
        pdf.close()
        
        cust = re.findall(r'(?s)Destination\n(.*?)\n[^\S\r\n]*SG-', text) #MV3
                
        dic_cust={'M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant MITSUI ELASTOMERS SINGAPORE PTE LTD 1 Ayer Chawan Place  701 AYER MERBAU ROAD SG SG Singapore': 'MITSUI ELASTOMERS SINGAPORE PTE LTD', 
          'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal BASF SOUTH EAST ASIA PTE LTD PENJURU ROAD 59 20 BANYAN AVENUE JURONG ISLAND SG SG Singapore': 'BASF SOUTH EAST ASIA PTE LTD', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal THE POLYOLEFIN -PULAU AYER MERBAU PENJURU ROAD 59 Ayer Merbau Road 401 SG SG Singapore': 'THE POLYOLEFIN -PULAU AYER MERBAU', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal MEGACHEM MANUFACTURING PTE LTD PENJURU ROAD 59 PIONEER ROAD 132 SG SG-639588 SINGAPORE': 'MEGACHEM MANUFACTURING PTE LTD', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal CHAMPIONX SG SERVICE PTE LTD PENJURU ROAD 59 11 AYER MERBAU ROAD, JURONG ISLAND SG SG-627526 SINGAPORE': 'CHAMPIONX SG SERVICE PTE LTD', 
          'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal CLP INTERNATIONAL PTE LTD PENJURU ROAD 59 2 TANJONG PENJURU SG SG-609017 SINGAPORE': 'CLP INTERNATIONAL PTE LTD', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal EXXONMOBIL CHEMICAL COMPANY PENJURU ROAD 59 JURONG ISLAND HIGHWAY 100 SG SG-627867 SINGAPORE': 'EXXONMOBIL CHEMICAL COMPANY', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal A.I.M. CHEMICAL INDUSTRIES PTE LTD PENJURU ROAD 59 19 TRACTOR ROAD SG SG-627977 SINGAPORE (DDP)': 'A.I.M. CHEMICAL INDUSTRIES PTE LTD', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal SUMITOMO CHEMICAL ASIA PTE LTD PENJURU ROAD 59 AYER MERBAU ROAD 700 SG SG-627525 SINGAPORE': 'SUMITOMO CHEMICAL ASIA PTE LTD', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD PENJURU ROAD 59 1 ANGSANA DRIVE JURONG ISLAND SG SG Singapore': 'ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD', 
          'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal CLP INTERNATIONAL PTE LTD PENJURU ROAD 59 2 TANJONG PENJURU SG SG-609017 SINGAPORE': 'CLP INTERNATIONAL PTE LTD', 
          'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal MSD INTERNATIONAL GMBH (SINGAPORE BRANCH) PENJURU ROAD 59 50 TUAS WEST DRIVE SG SG-638408 SINGAPORE': 'MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal SINGAPORE ADHESION PLANT PENJURU ROAD 59 100 JURONG ISLAND HIGHWAY SG SG Singapore': 'SINGAPORE ADHESION PLANT', 
          'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG PENJURU ROAD 59 TEL : 68676725 / 65342276 SG SG Singapore': 'SUMITOMO SEIKA-17', 
          'M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant SUMITOMO CHEMICAL ASIA PTE LTD 1 Ayer Chawan Place  AYER MERBAU ROAD 700 SG SG-627525 SINGAPORE': 'SUMITOMO CHEMICAL ASIA PTE LTD', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal MITSUI ELASTOMERS SINGAPORE PTE LTD PENJURU ROAD 59 701 AYER MERBAU ROAD SG SG Singapore': 'MITSUI ELASTOMERS SINGAPORE PTE LTD',
          'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal TOTALENERGIES MARKETING ASIA-PACIFIC MIDDLE EAST PTE LTD PENJURU ROAD 59 (LOBP) SG TUAS SOUTH AVENUE 14 2': 'TOTALENERGIES MARKETING ASIAPACIFIC',
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal BERTSCHI SINGAPORE PTE. LTD. PENJURU ROAD 59 SERAYA PLACE, JURONG ISLAND 2 SG SG Singapore': 'BERTSCHI SINGAPORE PTE. LTD.',
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal THE POLYOLEFIN - PULAU AYER MERBAU PENJURU ROAD 59 STREET 2 GATE NO 5 401 SG SG Singapore': 'THE POLYOLEFIN - PULAU AYER MERBAU G5',
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal THE POLYOLEFIN - PULAU AYER MERBAU PENJURU ROAD 59 STREET 2 GATE NO 23 401 SG SG Singapore': 'THE POLYOLEFIN - PULAU AYER MERBAU G23'
                 }
        custy = " ".join(line.strip() for line in cust[0].splitlines())
        cust_renam = dic_cust.get(custy)
                
        try:
            ship_num = regexp.search(text).group(1) #
            ship_num = ".".join(re.findall(r'\d+', ship_num))
            f1 = f.rpartition('/')[0] + '/'+ cust_renam + '_' + ship_num +'.pdf'
            os.rename(f, f1)
        except:
            pass
        try:
            ship_num = regexp_2.search(text).group(1) #
            f1 = f.rpartition('/')[0] + '/'+ cust_renam + '_' + ship_num +'.pdf'
            os.rename(f, f1)
        except:
            pass

In [4]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.serialization import AzureJSONEncoder
#from azure.ai.formrecognizer import AnalyzeResult
import json

endpoint = "https://...cognitiveservices.azure.com/"
key = ""

path_dir = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/"
path_destination = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Json/"
i = 1

document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key))

list_of_files = glob.glob(path_dir+'*.pdf')
for file in list_of_files:
    file = str(file)
    path_to_sample_documents = file
    with open(path_to_sample_documents, "rb") as f:
        poller = document_analysis_client.begin_analyze_document(
            "prebuilt-document", document=f
        )
    result = poller.result()

    analyze_result_dict = result.to_dict()
    with open(path_destination + 'data' + "_" +str(i)+ ".json", 'w') as f:
        json.dump(analyze_result_dict, f, cls=AzureJSONEncoder)
    i += 1


In [1]:
from azure.core.credentials import AzureKeyCredential #1) using Recognizer key_value pairs 18/2/2023!
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.serialization import AzureJSONEncoder
import json
import shutil
import glob, re, os
import pandas as pd

endpoint = "https://...cognitiveservices.azure.com/"
key = ""

#path_pdf = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/"
#path_json = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Json/"
path_pdf = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_split/KeyValue/"
path_json = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_split/Json/"
#path_json = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_split/Json_1/"
i=0
lst_trans = []
lst_dfq = []
field_lst = ['Shipping Number (BOL)', 'Comment', 'Transport No.', 'Origin', 'SAP Order No.', 'Article description']#MV2
field_lst2 = ['Shipping Number (BOL)', 'comment:', 'Transport No.', 'Origin', 'SAP Order No.', 'Article description']
field_lst5 = ['Shipping Number (BOL)', 'Transport comment:', 'Transport No.', 'Origin', 'SAP Order No.', 'Article description']
field_lst3 = ['Shipping Number (BOL)', 'Comment', 'Transport No.', 'SAP Order No.', 'Article description']
field_lst4 = ['Shipping Number (BOL)', 'Transport No.', 'SAP Order No.', 'Origin'] #basf
field_lst5 = ['Shipping Number (BOL)', 'Transport comment:', 'Transport No.', 'Origin', 'SAP Order No.', 'Article description']
field_lst6 = ['Shipping Number (BOL)', 'Transport No.', 'SAP Order No.', 'Article description'] #exxon

cust_lst = ['CHAMPIONX SG SERVICE PTE LTD', 'MITSUI ELASTOMERS SINGAPORE PTE LTD', 'MEGACHEM MANUFACTURING PTE LTD', 'CLP INTERNATIONAL PTE LTD', 
            'EXXONMOBIL CHEMICAL COMPANY', 'A.I.M. CHEMICAL INDUSTRIES PTE LTD', 'SUMITOMO CHEMICAL ASIA PTE LTD', 
            'SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG', 'BASF SOUTH EAST ASIA PTE LTD', 'THE POLYOLEFIN - PULAU AYER MERBAU', 
            'MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)', 'SINGAPORE ADHESION PLANT', 'ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD',
            'BERTSCHI SINGAPORE PTE. LTD.', 'TOTALENERGIES MARKETING ASIAPACIFIC', 'TOTALENERGIES MARKETING ASIA- PACIFIC MIDDLE EAST PTE LTD (LOBP)',
            'THE POLYOLEFIN - PULAU AYER MERBAU G5',
            'THE POLYOLEFIN - PULAU AYER MERBAU G23']

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)
list_of_pdf = glob.glob(path_pdf +'*.pdf')
for file_pdf in list_of_pdf:
    file = str(file_pdf)
    path_to_sample_documents = file
    with open(path_to_sample_documents, "rb") as f:
        poller = document_analysis_client.begin_analyze_document(
            "prebuilt-document", document=f, locale="en-US"
        )
        result = poller.result()
        
        lst_trans = []
        for kv_pair in result.key_value_pairs:
            if kv_pair.key and kv_pair.value:
                lst_trans.extend([[kv_pair.key.content, kv_pair.value.content]]) #result.key_value_pairs is list
                #print("Key '{}': Value: '{}'".format(kv_pair.key.content, kv_pair.value.content))
            else:
                #print("Key '{}': Value:".format(kv_pair.key.content))
                print('--------------------')
        analyze_result_dict = result.to_dict()
        
        qq = pd.DataFrame(lst_trans, columns=['SN', 'Va'])
        qqq = qq.set_index('SN').T.reset_index(drop=True)
        
        with open(path_json + 'data' + "_" +str(i)+ ".json", 'w') as f:
            json.dump(analyze_result_dict, f, cls=AzureJSONEncoder)             
        
        if 'Origin' in qqq.columns:
            try:
                q = qqq[field_lst]
            except:
                pass
            try:
                q = qqq[field_lst2]
            except:
                pass
            try:
                q = qqq[field_lst5]
            except:
                pass
            
            try:
                found = list(map(lambda x: x.startswith('BASF SOUTH EAST ASIA'), qqq['Destination']))
                #print(found[0])
                if found[0] == True:
                    try:
                        q = qqq[field_lst4]                     
                    except:
                        pass
            except:
                pass
            try:
                found_total = list(map(lambda x: x.startswith('TOTALENERGIES'), qqq['Destination']))
                if found_total[0] == True:
                    try:
                        q = qqq[field_lst1]
                    except:
                        pass
            except:
                pass
              
        if 'Origin' not in qqq.columns:
            try:
                q = qqq[field_lst3]
            except:
                pass
            try:
                q = qqq[field_lst4]
            except:
                pass
            try:
                q = qqq[field_lst6]
            except:
                pass
        q = q.loc[:, ~q.columns.duplicated()].copy() #remove dup cols
        #q['MV2'] = q['Origin'].map(chawanVopak_dic)
        #q['MV2'] = q['DEPOT_MV2'].apply(lambda x:chawanVopak_dic[x]) #map to new col with dict
        
        try:
            q.rename(columns={'Comment': 'Order Remarks', 'Transport No.': 'Transport No', 'Origin': 'DEPOT_MV2', 
                              'SAP Order No.': 'SAP Order No', 'Article description': 'PRODUCT', 'Shipping Number (BOL)': 'Cust Ref'}, 
                     inplace=True)
        except:
            q.rename(columns={'comment:': 'Order Remarks', 'Transport No.': 'Transport No', 'Origin': 'DEPOT_MV2', 
                              'SAP Order No.': 'SAP Order No', 'Article description': 'PRODUCT', 'Shipping Number (BOL)': 'Cust Ref'}, 
                     inplace=True)
        try:
            q.rename(columns={'Transport comment:': 'Order Remarks', 'Transport No.': 'Transport No', 'Origin': 'DEPOT_MV2', 
                              'SAP Order No.': 'SAP Order No', 'Article description': 'PRODUCT', 'Shipping Number (BOL)': 'Cust Ref'}, 
                     inplace=True)
        except:
            pass
        try:
            q.rename(columns={'Transport comment:': 'Order Remarks', 'Transport No.': 'Transport No', 'Origin': 'DEPOT_MV2', 
                              'SAP Order No.': 'SAP Order No', 'Product Information': 'PRODUCT', 'Shipping Number (BOL)': 'Cust Ref'}, 
                     inplace=True) #basf
        except:
            pass
        
        col_reord = ['Order Remarks', 'Cust Ref', 'Transport No', 'DEPOT_MV2', 'SAP Order No', 'PRODUCT']
        col_reord2 = ['Order Remarks', 'Cust Ref', 'Transport No', 'SAP Order No', 'PRODUCT']
        col_reord3 = ['Cust Ref', 'Transport No', 'SAP Order No', 'PRODUCT'] #exxon
        col_reord4 = ['Transport No', 'DEPOT_MV2', 'SAP Order No'] #basf
        
        if 'Order Remarks' not in q.columns:
            try:
                      new_cols = col_reord3 + (q.columns.drop(col_reord3).tolist())
            except:
                      pass
        if 'Order Remarks' in q.columns:
            try:
                new_cols = col_reord + (q.columns.drop(col_reord).tolist())                
            except:
                pass
            try:
                new_cols = col_reord2 + (q.columns.drop(col_reord2).tolist())                
            except:
                    pass
        if ('Order Remarks' not in q.columns) and ('Destination' in qqq.columns):
            new_cols = col_reord4 + (q.columns.drop(col_reord4).tolist())            
          
        q = q[new_cols]        
    lst_dfq.append(q)
    i += 1
q_trans = pd.concat(lst_dfq)
#q_trans = q_trans.dropna(subset=['DEPOT_MV2'])
q_trans.drop_duplicates(keep='first', inplace=True)

--------------------
--------------------
--------------------
--------------------
--------------------


In [2]:
q_trans

SN,Order Remarks,Cust Ref,Transport No,SAP Order No,PRODUCT,DEPOT_MV2
0,General Text,1003989188,6108374159,12737491,ACTREL 3356L AP VL,T1796 VOPAK TERMINAL - SINGAPO SG01
0,"General Text TIFU7789379, TIFU7789384, TIFU778...",1003999783,6108447400,5983129,SOLVESSO 150 AP VL\n<b>Delivery Time</b><br>BU...,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...
0,HOURS<br><br><b>Vehicle/Vessel</b><br>TIFU 152...,1003999602,6108445802,12758292,EXXSOL HEXANE SP AP VL\n<b>Delivery Time</b><b...,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...
0,"General Text PCVU 2630873, PCVU 2630889",1003999826,6108447693,12759254,EXXSOL HEPTANE AP VL\n<b>Delivery Time</b><br>...,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...


In [3]:
import numpy as np
q_trans_2 = q_trans.copy()
try:
    q_trans_2['DEPOT_MV2'] = q_trans_2['DEPOT_MV2'].replace(np.nan, 'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal')
except:
    pass
col_re = ['Order Remarks', 'Cust Ref', 'Transport No', 'DEPOT_MV2', 'SAP Order No', 'PRODUCT']
col_re_1 = ['Order Remarks', 'Cust Ref', 'Transport No', 'DEPOT_MV2', 'SAP Order No']

try:
    new_col = col_re + (q_trans_2.columns.drop(col_re).tolist())
    q_trans_2 = q_trans_2[new_col]
except:
    #q_trans_2['Cust Ref'] = ''
    q_trans_2['PRODUCT'] = ''
    q_trans_2['Order Remarks'] = ''
    q_trans_2['DEPOT_MV2'] = ''
    q_trans_2['DEPOT_MV2'] = q_trans_2['DEPOT_MV2'].replace(np.nan, 'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal')
    q_trans_2['DEPOT_MV2'] = q_trans_2['DEPOT_MV2'].replace('', 'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal')
    new_col = col_re + (q_trans_2.columns.drop(col_re).tolist())
    q_trans_2 = q_trans_2[new_col]
    
q_trans_2

SN,Order Remarks,Cust Ref,Transport No,DEPOT_MV2,SAP Order No,PRODUCT
0,General Text,1003989188,6108374159,T1796 VOPAK TERMINAL - SINGAPO SG01,12737491,ACTREL 3356L AP VL
0,"General Text TIFU7789379, TIFU7789384, TIFU778...",1003999783,6108447400,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,5983129,SOLVESSO 150 AP VL\n<b>Delivery Time</b><br>BU...
0,HOURS<br><br><b>Vehicle/Vessel</b><br>TIFU 152...,1003999602,6108445802,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,12758292,EXXSOL HEXANE SP AP VL\n<b>Delivery Time</b><b...
0,"General Text PCVU 2630873, PCVU 2630889",1003999826,6108447693,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,12759254,EXXSOL HEPTANE AP VL\n<b>Delivery Time</b><br>...


In [4]:
chawanVopak_dic = {'M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant 1 Ayer Chawan Place SG SG-627871 SINGAPORE': 'EXX004', 
                            'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal PENJURU ROAD ': 'EXX003',
                  'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal PENJURU ROAD': 'EXX003',
                  'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal PENJURU ROAD': 'EXX003',
                  'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal\nPENJURU ROAD 59\nSG\nSG-609142 SINGAPORE': 'EXX003',
                  'VOPAK': 'EXX003'}
for i in range(len(q_trans_2)):
    depot_mv2 = q_trans_2.iat[i, 3]
    try:
        depot_mv22 = depot_mv2.split(' ')[1]
        q_trans_2.iat[i, 3] = depot_mv22
    except:
        pass
    #first, last = str(depot_mv2).strip().splitlines()
    #depot_mv22 = depot_mv2.replace(last, '').replace('\n', ' ')  
q_trans_2['MV2'] = q_trans_2['DEPOT_MV2'].map(chawanVopak_dic)
try:
    q_trans_2['Order Remarks'] = q_trans_2['Order Remarks'].str.split('Vessel</b><br>').str[1]
except:
    pass
q_trans_2

SN,Order Remarks,Cust Ref,Transport No,DEPOT_MV2,SAP Order No,PRODUCT,MV2
0,NaN,1003989188,6108374159,VOPAK,12737491,ACTREL 3356L AP VL,EXX003
0,NaN,1003999783,6108447400,VOPAK,5983129,SOLVESSO 150 AP VL\n<b>Delivery Time</b><br>BU...,EXX003
0,"TIFU 1526380, TIFU 1526396,",1003999602,6108445802,CHAWAN,12758292,EXXSOL HEXANE SP AP VL\n<b>Delivery Time</b><b...,NaN
0,NaN,1003999826,6108447693,VOPAK,12759254,EXXSOL HEPTANE AP VL\n<b>Delivery Time</b><br>...,EXX003


In [5]:
#import shutil #2) parse json, merge df rocks 8/2/2023!
#import json, glob, re, os

i=0
lst_pp = []
lst_dpp = []
#path_json = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Json/"
path_json = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_split/Json/"
pp = pd.DataFrame(columns=['LOADING_PLANT_MV3', 'Coll Date'])
p_trans = pd.DataFrame(columns=['LOADING_PLANT_MV3', 'Coll Date'])

list_of_json = glob.glob(path_json + '*.json') 
for file_json in list_of_json:
    path_to_json = file_json
    with open(path_to_json) as file_object:
        data = json.load(file_object) #parse json file to Python obj

        z = data['content']
                
        cust = re.findall(r'(?s)Destination\n(.*?)\n[^\S\r\n]*SG-', z) #MV3
        #print(cust[0])    
        if cust[0] in cust_lst:
            dest_cx = cust[0]
            print(dest_cx)
            deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z) #
            deli_date = deli_date[0].split(' ', 1)[0]
            #print(deli_date)            
        if re.search("CHAMPIONX SG SERVICE PTE LTD", cust[0]):
            try:
                first, *others, last = cust[0].strip().splitlines()
                dest_cx = re.search(r"^[^\d]*", last, re.IGNORECASE).group() #get string before digits
                print(dest_cx)
            except:
                pass
        if cust[0] == 'EXXONMOBIL CHEMICAL COMPANY':
            deli_date = deli_date.split('\n')[0]
            #print(deli_date)
        if re.search("CHAMPIONX SG SERVICE PTE LTD", cust[0]):
            try:
                #deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]* ', z)
                deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z, re.MULTILINE) #
                deli_datex = deli_date[0].split(' ')[3]
                print(deli_datex)                
            except:
                deli_datex = re.findall(r'(?s)MY-(.*?)\n[^\S\r\n]*Measurements', z, re.MULTILINE) #
                             
        try:
            sap_num = re.findall(r'(?s)SAP Order No.\n(.*?)\n[^\S\r\n]*Order type', z)
            sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
            print(sap_num)          
        except:
            pass
        if cust[0] == "CHAMPIONX SG SERVICE PTE LTD":
            sap_num = re.findall(r'(?s)SG\n(.*?)\n[^\S\r\n]*normalorder', z)[0]
            #sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
            sap_num = re.findall("\d+", sap_num)[0] #get only digits
            print(sap_num)
        if cust[0] == "MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)":
            try:
                sap_num = re.findall(r'(?s)SAP Order No.\n(.*?)\n[^\S\r\n]*Order type', z)
                sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
                print(sap_num)                
            except:
                sap_num = re.findall(r'(?s)SAP Order No.(.*?)\n[^\S\r\n]*Items', z)
                sap_num = re.findall(r'(?s)SG(.*?)\n[^\S\r\n]*Delivery No.', sap_num[0], re.MULTILINE)
                #print(sap_num[0].strip())
                first, second, *others = sap_num[0].strip().splitlines()
                sap_num = first
                print(sap_num)
                
        try:
            shipping_num = re.findall(r'(?s)Order type\n(.*?)\n[^\S\r\n]*Shipping ', z)[0] #
            shipping_num = ".".join(re.findall(r'\d+', shipping_num))
        except:
            pass
        try:
            shipping_num = re.search('\nnormalorder\n(\d+)', z, re.IGNORECASE).group(1)
        except:
            pass
        if cust[0] == "SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG":
            try:
                regexp = re.compile(r'(?s)Order type (.*?)\n[^\S\r\n]*Shipping ', re.MULTILINE)
                shipping_num = regexp.search(z).group(1) #
                shipping_numseika = ".".join(re.findall(r'\d+', shipping_num))
            except: 
                pass
    if (re.search("CHAMPIONX SG SERVICE PTE LTD", cust[0]) and deli_date ==''):
        lst_pp.extend([[sap_num, dest_cx, deli_datex, shipping_num]])
    elif re.search("CHAMPIONX SG SERVICE PTE LTD", cust[0]):
        deli_date = deli_date[0].split(' ')[0]
        lst_pp.extend([[sap_num, dest_cx, deli_date, shipping_num]])
    else:
        lst_pp.extend([[sap_num, dest_cx, deli_date, shipping_num]])
    
    pp = pd.DataFrame(lst_pp)
    lst_dpp.append(pp)
    #shutil.move(file_json, path_json_1)
p_trans = pd.concat(lst_dpp)

# Rename multiple cols by Index
p_trans.rename(columns={p_trans.columns[0]: 'SAP Order No', p_trans.columns[1]: 'LOADING_PLANT_MV3', p_trans.columns[2]: 'Coll Date', p_trans.columns[3]: 'Cust_Ref'},inplace=True)
p_trans['Coll Date'] = p_trans['Coll Date'].str.split('\n').str[0]
p_trans['DELIVERY'] = p_trans['Coll Date']

p_trans[~p_trans.index.duplicated()] #drop dup rows by index
p_trans = p_trans.drop_duplicates(subset='SAP Order No', keep='first') #drop dup rows by col
p_trans

CLP INTERNATIONAL PTE LTD
12737491
CHAMPIONX SG SERVICE PTE LTD
10:30
20,000
5983129
5983129
MITSUI ELASTOMERS SINGAPORE PTE LTD
12758292
MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)
12759254
12759254


,SAP Order No,LOADING_PLANT_MV3,Coll Date,Cust_Ref,DELIVERY
0,12737491,CLP INTERNATIONAL PTE LTD,2023-02-02,1003989188,2023-02-02
1,5983129,CHAMPIONX SG SERVICE PTE LTD,2023-03-06,1003999783,2023-03-06
2,12758292,MITSUI ELASTOMERS SINGAPORE PTE LTD,2023-03-10,1003999602,2023-03-10
3,12759254,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH),2023-03-07,1003999826,2023-03-07


In [6]:
#import numpy as np
df_trans = pd.merge(p_trans, q_trans_2, left_on='SAP Order No', right_on='SAP Order No') #merge on common cols #how='left'
df_trans.drop_duplicates(subset='SAP Order No', keep='first', inplace=True)
df_trans[~df_trans.index.duplicated()] #drop dup rows
cols_reo = ['Cust Ref', 'Order Remarks', 'Transport No', 'Coll Date', 'DEPOT_MV2', 'LOADING_PLANT_MV3', 'DELIVERY', 
            'SAP Order No', 'PRODUCT'] #MV2
cols_reo_1 = ['Cust Ref', 'Order Remarks', 'Transport No', 'Coll Date', 'DEPOT_MV2', 'LOADING_PLANT_MV3', 'DELIVERY', 
            'SAP Order No', 'PRODUCT']
try:
    df_trans = df_trans[cols_reo]
except:
    pass
try:
    #df_trans['Cust Ref'] = ''
    #df_trans['PRODUCT'] = ''
    df_trans = df_trans[cols_reo_1]
except:
    pass
basf_dict = {'BASF SOUTH EAST ASIA PTE LTD': 'EXXAL 8 ROP VL'} #basf product
basf_dict_lst= list(basf_dict.keys())
df_trans['PRODUCT'] = np.where(df_trans.LOADING_PLANT_MV3==basf_dict_lst[0], 'EXXAL 8 ROP VL', df_trans['PRODUCT'])

df_trans.sort_values(["LOADING_PLANT_MV3", "SAP Order No"],
               axis=0, ascending=True,
               inplace=True,
               na_position="first") #sorting function multiple col

df_trans['Order Remarks'] = df_trans['Order Remarks'].astype('str')
for i in range (len(df_trans)):
    try:
        found_rem = df_trans.at[df_trans.index[i+1], 'Order Remarks']
    except:
        pass
    if (found_rem.startswith('<b>Delivery Time</b>') or found_rem.startswith('<b>Delivery Time</b><br>BUSINESS HOURS')) == True:
        try:
                        found_rem = found_rem.split('Vessel</b><br>')[1]
                        df_trans.iat[i, 1] = found_rem                        
        except:
                        pass
df_trans['Cust Ref'].fillna(0, inplace=True)
df_trans.index = np.arange(1, len(df_trans)+1)
df_trans

,Cust Ref,Order Remarks,Transport No,Coll Date,DEPOT_MV2,LOADING_PLANT_MV3,DELIVERY,SAP Order No,PRODUCT
1,1003999783,nan,6108447400,2023-03-06,VOPAK,CHAMPIONX SG SERVICE PTE LTD,2023-03-06,5983129,SOLVESSO 150 AP VL\n<b>Delivery Time</b><br>BU...
2,1003989188,nan,6108374159,2023-02-02,VOPAK,CLP INTERNATIONAL PTE LTD,2023-02-02,12737491,ACTREL 3356L AP VL
3,1003999602,"TIFU 1526380, TIFU 1526396,",6108445802,2023-03-10,CHAWAN,MITSUI ELASTOMERS SINGAPORE PTE LTD,2023-03-10,12758292,EXXSOL HEXANE SP AP VL\n<b>Delivery Time</b><b...
4,1003999826,nan,6108447693,2023-03-07,VOPAK,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH),2023-03-07,12759254,EXXSOL HEPTANE AP VL\n<b>Delivery Time</b><br>...


In [7]:
from datetime import datetime
df_trans['DELIVERY'] = pd.to_datetime(df_trans['DELIVERY'], format="%Y-%m-%d")
curr_filter = datetime.now()
df_trans = df_trans.loc[df_trans['DELIVERY'] >= curr_filter]

df_trans['DELIVERY'] = df_trans['DELIVERY'].apply(str)
#df_trans.dtypes

C:\Users\john.tan\AppData\Local\Temp\ipykernel_34536\3363213041.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans['DELIVERY'] = df_trans['DELIVERY'].apply(str)


In [8]:
for i in range (len(df_trans)):
    if df_trans.apply(lambda x: (x['Order Remarks'] in ['nan']), axis=1).any() == True:
        df_trans.at[i, 'Order Remarks'] = 'No transports comments'
df_trans = df_trans.dropna(subset=['Cust Ref'])
df_trans

,Cust Ref,Order Remarks,Transport No,Coll Date,DEPOT_MV2,LOADING_PLANT_MV3,DELIVERY,SAP Order No,PRODUCT
1,1003999783,No transports comments,6108447400,2023-03-06,VOPAK,CHAMPIONX SG SERVICE PTE LTD,2023-03-06 00:00:00,5983129,SOLVESSO 150 AP VL\n<b>Delivery Time</b><br>BU...
3,1003999602,"TIFU 1526380, TIFU 1526396,",6108445802,2023-03-10,CHAWAN,MITSUI ELASTOMERS SINGAPORE PTE LTD,2023-03-10 00:00:00,12758292,EXXSOL HEXANE SP AP VL\n<b>Delivery Time</b><b...
4,1003999826,nan,6108447693,2023-03-07,VOPAK,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH),2023-03-07 00:00:00,12759254,EXXSOL HEPTANE AP VL\n<b>Delivery Time</b><br>...


In [26]:
import openpyxl
import datetime
with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/KNS/For_ChiaMean.xlsx", mode='a', if_sheet_exists='replace', engine='openpyxl') as writer8:
    sheetName = 'NewTransports_' + datetime.datetime.now().strftime("%Y%m%d %H%M")
    df_trans.to_excel(writer8, sheet_name=sheetName, engine='openpyxl')
    worksheet8 = writer8.sheets[sheetName]
    writer8.save()

C:\Users\john.tan\AppData\Local\Temp\ipykernel_5888\1951531804.py:7: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer8.save()


In [15]:
import re #parse json files 23/1/2023!
from re import search

path_dir = 'C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Json/'
cols = ['Cust Ref', 'Order Remarks', 'Transport No', 'Coll Date', 'DEPOT_MV2', 'PRODUCT', 'LOADING_PLANT_MV3', 'DELIVERY', 'SAP Order No']
df_trans = pd.DataFrame(columns=cols)
#df_trans.columns = cols
lst_trans = []
i = 0

os.chdir(path_dir)
for file in glob.glob("*.json"):
    file = str(file)
    with open(path_dir+file) as file_object:
        data = json.load(file_object) #parse json file to Python obj

    z = data['content']

    trans_num = re.search(r'Transport No. (.*?) Delivery', z).group(1) #extract str between two strings
    #trans_ID = re.search(r'ID (.*?) Weight', z, flags=re.MULTILINE).group(1)
    #weight = re.search(r'Weight (.*?)kg', z).group(1) #
    cust = re.findall(r'(?s)Destination\n(.*?)\n[^\S\r\n]*SG-', z) #MV3
    print(cust[0])
    try:
            sap_num = re.findall(r'(?s)SAP Order No.\n(.*?)\n[^\S\r\n]*Order type', z)
            sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
            print(sap_num)
    except:
            pass
    
    if (cust[0] == 'SINGAPORE ADHESION PLANT' and (isinstance(sap_num, list) == True)):
        try:
                sap_adh = re.findall(r"(\d+)\nOrder type", z)[0]
                print('SAP1')
                print(sap_adh)
        except:
                sap_numbr = re.search('SAP Order No.\n(\d+)', z)
                if sap_numbr:
                    print('SAP2')
                    sap_adh = sap_numbr.group(1)
                    print(sap_adh)           
                
    if cust[0] == "CHAMPIONX SG SERVICE PTE LTD":
        sap_num = re.findall(r'(?s)SG\n(.*?)\n[^\S\r\n]*normalorder', z)[0]
        #sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
        print(sap_num)
    try:
        shipper = re.search(r'(.*?) Scheduling', z).group(1) #
        shipping_num = re.search('\nnormalorder\n(\d+)', z, re.IGNORECASE).group(1) ## get numbers after str

    except:
        shipper = re.findall(r'(?s)Origin\n(.*?)\n[^\S\r\n]*SG', z)[0]       
    try:
        prod_info = re.findall(r'(?s)SAP Order No.\n(.*?)\n[^\S\r\n]*UTC', z) #
        first, *others, last = prod_info[0].strip().splitlines()
        prod_info = last
        #print(last)
    except:
        prod_info = re.findall(r'(?s)Product information\n(.*?)\n[^\S\r\n]*Loading', z) #
        
    if search("CHAMPIONX SG SERVICE PTE LTD", cust[0]):
        try:
            #deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]* ', z)
            deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z, re.MULTILINE) #
            deli_datex = deli_date[0].split(' ')[3]
            #print(deli_datex)
        except:
            deli_datex = re.findall(r'(?s)MY-(.*?)\n[^\S\r\n]*Measurements', z, re.MULTILINE) #
    
    if cust[0] == "MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)":
        try:
            sap_num = re.findall(r'(?s)SAP Order No.\n(.*?)\n[^\S\r\n]*Order type', z)
            sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
            print(sap_num)
        except:
            sap_num = re.findall(r'(?s)SAP Order No.(.*?)\n[^\S\r\n]*Items', z)
            sap_num = re.findall(r'(?s)SG(.*?)\n[^\S\r\n]*Delivery No.', sap_num[0], re.MULTILINE)
            #print(sap_num[0].strip())
            first, second, *others = sap_num[0].strip().splitlines()
            sap_num = first
            print(sap_num)
        z_msd = z.replace('\n', '')    
        try:
            shipping_numMSD = re.search('SingaporeSG(\d+)', z_msd, re.IGNORECASE).group(1) #
        except:
            pass
        try:
            shipping_numMSD = re.search('\nSG\n(\d+)', z_msd, re.IGNORECASE).group(1)
        except:
            pass
        try:
            shipping_numMSD = shipping_numMSD[-15:]
            shipping_numsd = shipping_numMSD[0:10]
            print('z_msd '+shipping_numsd)
        except:
            pass
        
    if cust[0] == "SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG":
        try:
            regexp = re.compile(r'(?s)Order type (.*?)\n[^\S\r\n]*Shipping ', re.MULTILINE)
            shipping_num = regexp.search(z).group(1) #
            shipping_numseika = ".".join(re.findall(r'\d+', shipping_num))
        except: 
            pass
    
    po_num = re.findall(r'(?s)Purchase order No.\n(.*?)\n[^\S\r\n]*Gross Weight', z)
    cust = re.findall(r'(?s)Destination\n(.*?)\n[^\S\r\n]*SG-', z) #MV3
    origin = re.findall(r'(?s)Origin\n(.*?)\n[^\S\r\n]*SG-', z) #MV2
    plant = re.findall(r'(?s)Division\n(.*?)\n[^\S\r\n]*Product', z)
    #vehicle = re.search(r'AP - (.*?) T', z).group(1)
    #vehicle = str(vehicle) + 'Truck'
    
    #deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]* ', z)
    deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z) #
    deli_date = deli_date[0].split(' ', 1)[0]
    if cust[0] == 'EXXONMOBIL CHEMICAL COMPANY':
        deli_date = deli_date.split('\n')[0]
        
    if cust[0] == 'RESPACK MANUFACTURING SDN BHD':
        p = re.compile('MY-([\S\s]*)\n', re.MULTILINE) #Respack
        deli_date = re.findall(p, respack)[0].split('\n')[1].split(' ')[0]
    try:
        shipping_num = re.findall(r'(?s)Order type\n(.*?)\n[^\S\r\n]*Shipping ', z)[0] #
        shipping_num = ".".join(re.findall(r'\d+', shipping_num))
    except:
        pass
    try:
        shipping_num = re.search('\nnormalorder\n(\d+)', z, re.IGNORECASE).group(1)
    except:
        pass
    prod_info = re.findall(r'(?s)Product information\n(.*?)\n[^\S\r\n]*Loading', z) #
    
    try:
        deadline = re.search(r'Deadline (.*?) ', z).group(1)
        tifu_patt = r'(?<=General Text)(\s.+)?\nAdditional'
        tifu = re.findall(tifu_patt, z, re.DOTALL)[0]
        tifu = os.linesep.join([s for s in tifu.splitlines() if s])
        first, *others = tifu.splitlines()
    except:
        pass  
    try:
        ord_rem = cust[0] + prod_info[0]
    except:
        pass
    if search("CHAMPIONX SG SERVICE PTE LTD", cust[0]):
        z_cx = z.replace('\n', '')
        try:
            prod_info =  re.search(r'Article description(.*?)Comment', z_cx).group(1) ##regex between 2 strings
            ord_rem = cust[0]+prod_info
        except:
            prod_info = re.search(r'Article short description(.*?)Article description', z_cx) #
    if search('BASF SOUTH EAST ASIA PTE LTD', cust[0]):
        z_basf = z.replace('\n', '')
        try:
            prod_info =  re.search(r'Article description(.*?)Comment', z_basf).group(1) ##regex between 2 strings
            print('basf prod')
            print(prod_info)
            ord_rem = cust[0] + prod_info
        except:
            prod_info = re.search(r'Product information(.*?)Loading time', z_basf) # 
            try:
                ord_rem = cust[0] + prod_info
            except:
                pass
    
    if origin[0] =='M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant':
            df_trans.loc[i, 'MV2'] = 'EXX004'
    else:
        df_trans.loc[i, 'MV2'] = 'EXX003'
    if cust[0] == 'THE POLYOLEFIN - PULAU AYER MERBAU':
        try:
            cust_1 = cust[1].split('\n')[-1].split(', SG')[0]
            df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust_1
        except:
            pass
        try:
            cust_1 = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z) #THE POLYOLEFIN - PULAU AYER MERBAU
            df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust_1
        except:
            pass
    else: 
        df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust[0]
    
    try:
        df_trans.loc[i, 'Cust Ref'] = shipping_num
    except:
        pass
    try:
        df_trans.loc[i, 'Cust Ref'] = second
    except:
        pass
    if cust[0] == "MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)":
        try:
            df_trans.loc[i, 'Cust Ref'] = shipping_numsd
        except:
            df_trans.loc[i, 'Cust Ref'] = shipping_num
    
    if cust[0] == "SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG":
        try:
            df_trans.loc[i, 'Cust Ref'] = shipping_numseika
        except:
            df_trans.loc[i, 'Cust Ref'] = shipping_num
    try:
        df_trans.loc[i, 'Order Remarks'] = ord_rem
    except:
        pass
    df_trans.loc[i, 'Transport No'] = trans_num
    df_trans.loc[i, 'Coll Date'] = deli_date
    df_trans.loc[i, 'DEPOT_MV2'] = origin[0]
    try:
        df_trans.loc[i, 'PRODUCT'] = prod_info[0]
    except:
        df_trans.loc[i, 'PRODUCT'] = last
    if search("CHAMPIONX SG SERVICE PTE LTD", cust[0]):
        df_trans.loc[i, 'PRODUCT'] = prod_info
    
    df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust[0]
    df_trans.loc[i, 'DELIVERY'] = deli_date
    try:
        df_trans.loc[i, 'SAP Order No'] = sap_num
    except:
        pass
    if (cust[0] == 'SINGAPORE ADHESION PLANT' and (isinstance(sap_num, list) == True)):
        try:
            df_trans.loc[i, 'SAP Order No'] = sap_adh
        except:
            pass
    try:
        if cust[1] =="CHAMPIONX SG\nSERVICE PTE LTD":
            df_trans.loc[i, 'DELIVERY'] = deli_datex
            df_trans.loc[i, 'Coll Date'] = deli_datex
            df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust[1].replace('\n', ' ')
            df_trans.loc[i, 'DEPOT_MV2'] = origin[0].replace('Destination\n', '')
    except: pass
    i += 1

df_trans

A.I.M. CHEMICAL INDUSTRIES PTE LTD
12726055
ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD
12726059


,Cust Ref,Order Remarks,Transport No,Coll Date,DEPOT_MV2,PRODUCT,LOADING_PLANT_MV3,DELIVERY,SAP Order No,MV2
0,1003982065,A.I.M. CHEMICAL INDUSTRIES PTE LTDISOPAR L AP VL,6108265676,2023-01-03,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,ISOPAR L AP VL,A.I.M. CHEMICAL INDUSTRIES PTE LTD,2023-01-03,12726055,EXX003
1,1003982065,ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD...,6108298482,2023-01-18,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,EXXSOL HEXANE AP VL,ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD,2023-01-18,12726059,EXX003


In [10]:
df_trans_trans = df_trans.copy()

df_trans_trans['Coll Date'] = [x.split('\n')[0].replace(' ', '') for x in df_trans_trans['Coll Date']]
df_trans_trans['DELIVERY'] = [x.split('\n')[0].replace(' ', '') for x in df_trans_trans['DELIVERY']]
df_trans_trans['PRODUCT'] = [x.split('\n')[0].replace(' ', '') for x in df_trans_trans['PRODUCT']]
#s = '(2023-02-03\n18,000)'
#m = re.match('\(([\w\s]+)\n([\w\s]+)\)', s)
#coll_date = m.group(1)
#tim = m.group(2)
df_trans_trans['DELIVERY'] = df_trans_trans['DELIVERY'].str[0: -8]
df_trans = df_trans_trans.copy()
df_trans
#https://stackoverflow.com/questions/44922259/python-using-regex-to-extract-parts-of-a-string-in-pandas-column

,Cust Ref,Order Remarks,Transport No,Coll Date,DEPOT_MV2,LOADING_PLANT_MV3,DELIVERY,SAP Order No,PRODUCT
1,1003999783,No transports comments,6108447400,2023-03-06,VOPAK,CHAMPIONX SG SERVICE PTE LTD,20,5983129,SOLVESSO150APVL
3,1003999602,"TIFU 1526380, TIFU 1526396,",6108445802,2023-03-10,CHAWAN,MITSUI ELASTOMERS SINGAPORE PTE LTD,20,12758292,EXXSOLHEXANESPAPVL
4,1003999826,nan,6108447693,2023-03-07,VOPAK,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH),20,12759254,EXXSOLHEPTANEAPVL


In [12]:
basf_dict = {'BASF SOUTH EAST ASIA PTE LTD': 'EXXAL 8 ROP VL'}
basf_dict_lst= list(basf_dict.keys())
df_trans['PRODUCT'] = np.where(df_trans.LOADING_PLANT_MV3==basf_dict_lst[0], 'EXXAL 8 ROP VL', df_trans.PRODUCT)

df_trans['Cust Ref'].fillna(0, inplace=True)
#df_trans['LOADING_PLANT_MV3'].isin(basf_dict_lst)
df_trans.head()

,Cust Ref,Order Remarks,Transport No,Coll Date,DEPOT_MV2,LOADING_PLANT_MV3,DELIVERY,SAP Order No,PRODUCT
1,1003999783,No transports comments,6108447400,2023-03-06,VOPAK,CHAMPIONX SG SERVICE PTE LTD,2023-03-0600:00:00,5983129,SOLVESSO 150 AP VL\n<b>Delivery Time</b><br>BU...
3,1003999602,"TIFU 1526380, TIFU 1526396,",6108445802,2023-03-10,CHAWAN,MITSUI ELASTOMERS SINGAPORE PTE LTD,2023-03-1000:00:00,12758292,EXXSOL HEXANE SP AP VL\n<b>Delivery Time</b><b...
4,1003999826,nan,6108447693,2023-03-07,VOPAK,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH),2023-03-0700:00:00,12759254,EXXSOL HEPTANE AP VL\n<b>Delivery Time</b><br>...


In [14]:
temp = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Template_Local.csv") #template for upload
tables = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Tables.csv") #tables for mapping

In [15]:
temp_upload = pd.concat([temp, df_trans], ignore_index=True)

temp_upload = pd.merge(left=temp_upload, right=tables, how='left') #merge df 
temp_upload = temp_upload.fillna(method='ffill')
#temp_upload = temp_upload.drop(['MV3'], axis=1)
temp_upload = temp_upload.rename(columns={'Code': 'MV3'}) #for upload to TMS

In [16]:
colss = ['Customer No', 'Order Remarks', 'Delivery Type No', 'ISO abbreviation', 'Cargo Type No', 'MV1', 'MV2', 'MV4', 'MV5', 'MV6',
         'Plant Code', 'Special Handling No', 'Destination', 'MV3', 'Delivery Type']
temp_upload_1 = temp_upload.drop(columns=colss, axis=1)
temp_upload_1 = temp_upload_1.rename(columns={'LOADING_PLANT_MV3': 'LOADING PLANT', 'DEPOT_MV2': 'DEPOT'})
colss_reord = ['Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT', 'LOADING PLANT', 'DELIVERY', 'SAP Order No']
temp_upload_1 = temp_upload_1[colss_reord] #to compare with master
temp_upload_1 = temp_upload_1.reset_index().drop(columns='index')
#temp_upload_1

In [17]:
temp_upload_1['Coll Date'] = pd.to_datetime(temp_upload_1['Coll Date']).dt.strftime('%d/%m/%Y') #format datetime
temp_upload_1['DELIVERY'] = pd.to_datetime(temp_upload_1['DELIVERY']).dt.strftime('%d/%m/%Y')

temp_upload_1['SAP Order No'] = pd.to_numeric(temp_upload_1['SAP Order No'])
temp_upload_1.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
temp_upload_1.index = np.arange(1, len(temp_upload_1)+1)
temp_upload_1

ParserError: Unknown string format: 2023-03-0600:00:00 present at position 0

In [13]:
columns = ['NO', 'SH Reference', 'Job Type', 'Shipper', 'Permit', 'ETA Date', 'QTY', 'Cont Size', 'Cont Type',
          'Cont No', 'SEAL', 'TIME SLOT', 'REMARKS', 'TD- STATUS']
master = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/EM master file.csv")
master = master.drop(columns=columns, axis=1)
master = master.iloc[1:] #remove first row
master = master.reset_index().drop(columns='index')

from datetime import datetime
try:
    master['Coll Date'] = pd.to_datetime(master['Coll Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d') #format datetime
    master['DELIVERY'] = pd.to_datetime(master['DELIVERY'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
except:
    pass

master.drop(index=master.index[0], axis=1, inplace=True) #drop first row
master['EM SO# Ref'] = pd.to_numeric(master['EM SO# Ref'], errors='coerce') #convert non-numeric values to NaN
master.dropna(subset=['EM SO# Ref'], inplace=True) #drop rows with NaN in col
master['EM SO# Ref'] = master['EM SO# Ref'].astype(np.int64)
#master['EM SO# Ref'] = master['EM SO# Ref'].apply(pd.to_numeric)

In [14]:
temp_upload_cust = temp_upload_1['SAP Order No'].tolist()
temp_upload_cust = list(map(int, temp_upload_cust)) #convert list items to int

master = master[master['EM SO# Ref'].isin(temp_upload_cust)] #select rows based on list
#master = master[master['SAP Order No'].isin(temp_upload_cust)]
col_mas = ['Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT', 'LOADING PLANT', 'DELIVERY', 'EM SO# Ref'] #'EM SO# Ref'
master = master[col_mas]
master.rename(columns={'EM SO# Ref': 'SAP Order No'}, inplace=True)

master.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
master.index = np.arange(1, len(master)+1)

master['Cust Ref'].fillna(value=0, inplace=True)
master['Transport No'].fillna(value=0, inplace=True)
master['Cust Ref'] = master['Cust Ref'].astype('int64')
master['Transport No'] = master['Transport No'].astype('int64')
if len(master) == 0:
    print('\033[1m'+ '\033[94m'+ 'All download Transports are new')

master.loc[~master['Coll Date'].str.contains('[a-zA-Z]')] #remove rows with alphabets in col
master.loc[~master['DELIVERY'].str.contains('[a-zA-Z]')]
master
#https://stackoverflow.com/questions/69255495/remove-rows-with-alphabets-in-a-given-column

,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No
1,1003977817,6108295225,2023-01-11,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SPECTRASYN 4,TOTALENERGIES MARKETING ASIA-PACIFIC MIDDLE EA...,2023-01-11,12728630
2,1003994729,6108403829,2023-02-27,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SPECTRASYN 4,TOTALENERGIES MARKETING ASIA-PACIFIC MIDDLE EA...,2023-02-27,12750473


In [15]:
if len(master) != 0:
    master['Coll Date'] = pd.to_datetime(master['Coll Date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d') #format datetime
    master['DELIVERY'] = pd.to_datetime(master['DELIVERY'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')

In [16]:
from datetime import timedelta
columns = ['NO', 'SH Reference', 'Job Type', 'Shipper', 'Permit', 'ETA Date', 'QTY', 'Cont Size', 'Cont Type',
          'Cont No', 'SEAL', 'TIME SLOT', 'REMARKS', 'TD- STATUS']
df_sap = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/EM master file.csv")
#df_sap = master.copy() #transports with changes
df_sap = df_sap.drop(columns=columns, axis=1)
df_sap = df_sap.reset_index().drop(columns='index')

df_sap.rename(columns={'EM SO# Ref': 'SAP_Order_No'}, inplace=True)
sap = df_sap[df_sap.duplicated('SAP_Order_No', keep=False)].groupby('SAP_Order_No')['Coll Date'].apply(list).reset_index()
lst_sap = sap['SAP_Order_No'].tolist()
sap_1 = df_sap[df_sap["SAP_Order_No"].isin(lst_sap)]

curr = datetime.now()
coll_start = curr - timedelta(days=30)
#coll_end = curr + timedelta(days=14)

from simple_colors import *
if len(lst_sap) != 0:
    for i in range(len(sap)):
        coll_date = sap.at[i, 'Coll Date']
        coll_date_lst = [datetime.strptime(coll, "%d/%m/%Y").date() for coll in coll_date]
        coll_latest = max(coll_date_lst)#.strftime("%Y-%m-%d")
        
        if coll_latest > coll_start.date():
            print('\033[1m'+ '\033[34m' + 'Changes to Coll Date')
            print('SAP Order Number: '+ magenta(str(lst_sap[0]).strip(), ['bold', 'underlined'])+' '+green(str(max(coll_date_lst).strftime("%d/%m%Y")), ['bold', 'underlined']))
            print(sap.at[i, 'Coll Date'])       
else:
    print('No transports changes')

In [17]:
from datetime import datetime
for i in range(len(sap)):
    sap_order = sap.at[i, 'SAP_Order_No']
    coll_date = sap.at[i, 'Coll Date']
    coll_date_lst = [datetime.strptime(coll, "%d/%m/%Y").date() for coll in coll_date] #convert elem in list to date
    print('SAP Order Number: '+ str(sap_order).strip() + ' Latest update coll date: '+ max(coll_date_lst).strftime("%Y-%m-%d"))

SAP Order Number: 4452123248.0 Latest update coll date: 2021-04-30
SAP Order Number: 4452134496.0 Latest update coll date: 2021-05-31
SAP Order Number: 4452137410.0 Latest update coll date: 2021-05-31
SAP Order Number: 4452146793.0 Latest update coll date: 2021-06-24
SAP Order Number: 4452147393.0 Latest update coll date: 2021-06-29
SAP Order Number: 4452159155.0 Latest update coll date: 2021-08-27
SAP Order Number: 4452159194.0 Latest update coll date: 2021-07-29
SAP Order Number: 4452170447.0 Latest update coll date: 2021-08-30
SAP Order Number: 4452173164.0 Latest update coll date: 2021-08-30
SAP Order Number: 4452182433.0 Latest update coll date: 2021-09-30
SAP Order Number: 4452184599.0 Latest update coll date: 2021-09-23
SAP Order Number: 4452193791.0 Latest update coll date: 2021-10-29
SAP Order Number: 4452199566.0 Latest update coll date: 2021-10-27
SAP Order Number: 4452205742.0 Latest update coll date: 2021-11-29
SAP Order Number: 4452217805.0 Latest update coll date: 2021-1

In [18]:
df_sap_1 = df_sap.drop(df_sap.iloc[:, 9:15], axis=1)
df_sap_1.drop(df_sap_1.columns[0], axis=1, inplace=True) #drop first col
df_sap_1['SAP_Order_No'] = df_sap_1['SAP_Order_No'].apply(str)

try:
    df_sap_1[['SAP_Order_No', 'SAP_Order_No1']] = df_sap_1['SAP_Order_No'].str.split(' / ', 1, expand=True) #split col in 2
    df_sap_1 = df_sap_1.iloc[:, :-1] #drop last col
except:
    pass
df_sap_1['SAP_Order_No'] = pd.to_numeric(df_sap_1['SAP_Order_No'], errors='coerce') #convert non-numeric values to NaN
df_sap_1.dropna(subset=['SAP_Order_No'], inplace=True) #drop rows with NaN in col

C:\Users\john.tan\AppData\Local\Temp\ipykernel_37212\2429366599.py:6: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_sap_1[['SAP_Order_No', 'SAP_Order_No1']] = df_sap_1['SAP_Order_No'].str.split(' / ', 1, expand=True) #split col in 2


In [19]:
#new transports works 02/02/2023!
uploaded_trans = df_trans['SAP Order No'].tolist()
master_trans = df_sap_1['SAP_Order_No'].tolist()

uploaded_trans = np.unique(uploaded_trans).astype(float)
master_trans = np.unique(master_trans).astype(float)

#np.setdiff1d(uploaded_trans, master_trans).astype(int)
uploaded_trans = [x for x in uploaded_trans if x not in master_trans]
uploaded_trans = list(map(int, uploaded_trans)) #convert to int
uploaded_trans = [ele for ele in uploaded_trans if ele > 0]

if len(uploaded_trans) !=0:
    print('New transport SAP Order Number ')
    print('\033[1m'+ '\033[32m'+ str(uploaded_trans))
else:
    print('No new SAP Order Number')

new_trans = [] #convert list of int to str
for x in uploaded_trans:
    str_x = "% s" % x
    new_trans.append(str_x)

df_newtrans = df_trans[df_trans['SAP Order No'].isin(new_trans)]

df_newtrans.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
if len(df_newtrans) !=0:
    df_newtrans

No new SAP Order Number


C:\Users\john.tan\AppData\Local\Temp\ipykernel_37212\1146280252.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_newtrans.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')


In [74]:
#comp (iterate loading plant) with color, write to excel works 20/1/2023!
#import numpy as np

def highlight_diff(data, color='lightgreen'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('First', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

def repeatRows(d, n):
    return pd.concat([d]*n)

temp_upload_6 = temp_upload_1.copy()
master_6 = master.copy()
tp = temp_upload_6.groupby('LOADING PLANT')   
lst_tp = [tp.get_group(x) for x in tp.groups] #groupby and split df
ms = master_6.groupby('LOADING PLANT')
lst_ms = [ms.get_group(x) for x in ms.groups]

for i in range(min(len(lst_tp), len(lst_ms))):
    n = abs(len(lst_ms[i]) - len(lst_tp[i])) #ensure both df same len
    if n!= 0:

        if len(lst_ms[i]) > len(lst_tp[i]):
            lst_tp[i] = pd.concat([lst_tp[i], repeatRows(lst_tp[i][-1:], n)])
        else:
            lst_ms[i] = pd.concat([lst_ms[i], repeatRows(lst_ms[i][-1:], n)])
            
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        df_all = pd.concat([lst_tp[i], lst_ms[i]], axis='columns', keys=['First', 'Second'])
        df_comp = df_all.swaplevel(axis='columns')[lst_tp[i].columns[1:]]

        lst_ms[i] = lst_ms[i].dropna(subset=['Transport No','Coll Date'], how='all')
        lst_tp[i] = lst_tp[i].dropna(subset=['Transport No','Coll Date'], how='all')
            
        #lst_tp[i] = lst_tp[i].iloc[:, :-1]
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
            
        df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
        #df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)

        #print(df_comp)
        with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Output/comp.xlsx", mode="a", engine="openpyxl") as writer:
            df_comp.to_excel(writer, sheet_name='comp_'+ str(i) + datetime.now().strftime("%Y%m%d_%H%M"), engine='openpyxl')
        with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_Test/Comp_color/comp.xlsx", mode="a", engine="openpyxl") as writer1:
            df_comp.to_excel(writer1, sheet_name='comp_'+ str(i) + datetime.now().strftime("%Y%m%d_%H%M"), engine='openpyxl')

C:\Users\john.tan\AppData\Local\Temp\ipykernel_1056\2937698046.py:41: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
C:\Users\john.tan\AppData\Local\Temp\ipykernel_1056\2937698046.py:41: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
C:\Users\john.tan\AppData\Local\Temp\ipykernel_1056\2937698046.py:41: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
C:\Users\john.tan\AppData\Local\Temp\ipykernel_1056\2937698046.py:41: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keywor

In [ ]:
class MyList(list):
    def __contains__(self, typ):
        return any(isinstance(val, typ) for val in self) #optional

x = MyList(uploaded_trans)
print (int in x)

In [48]:
#optional
def repeatRows(d, n):
    return pd.concat([d]*n)

n = abs(len(master) - len(temp_upload_1)) #ensure both df same len
if n!= 0:
    if len(master) > len(temp_upload_1):
        temp_upload_1 = pd.concat([temp_upload_1, repeatRows(temp_upload_1[-1:], n)])
        temp_upload_1.index = np.arange(1, len(temp_upload_1)+1)
        #print(len(temp_upload_1))
    else:
        master = pd.concat([master, repeatRows(master[-1:], n)])
        master.index = np.arange(1, len(master)+1)
        #print(len(master))
else:
    print('No existing transports to compare')

In [24]:
df_all = pd.concat([temp_upload_1, master], axis='columns', keys=['First', 'Second'])
df_comp = df_all.swaplevel(axis='columns')[temp_upload_1.columns[1:]]

def highlight_diff(data, color='lightgreen'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('First', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

master = master.dropna(subset=['Transport No','Coll Date'], how='all')
temp_upload_1 = temp_upload_1.dropna(subset=['Transport No','Coll Date'], how='all')

df_comp = df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)
#df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)
df_comp

C:\Users\john.tan\AppData\Local\Temp\ipykernel_34000\2874637043.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)


In [25]:
master.compare(temp_upload_1, keep_shape=True, keep_equal=True)

Cust Ref             Transport No               Coll Date              \
          self       other         self       other        self       other   
1   1003991851  1003991851   6108388149  6108388149  2023-02-20  20/02/2023   
2   1003993137  1003993137   6108397639  6108397639  2023-02-23  23/02/2023   
3   1003993102  1003993102   6108397638  6108397638  2023-02-23  23/02/2023   
4   1003991504  1003991504   6108385217  6108385217  2023-02-03  03/02/2023   
5   1003989742  1003989742   6108377858  6108377858  2023-02-10  10/02/2023   
6   1003989797  1003989797   6108377859  6108377859  2023-02-14  14/02/2023   
7   1003991839  1003991839   6108388146  6108388146  2023-02-20  20/02/2023   
8   1003992819  1003992819   6108395796  6108395796  2023-02-14  14/02/2023   
9   1003992715  1003992715   6108395795  6108395795  2023-02-14  14/02/2023   
10  1003995866  1003995866   6108419506  6108419506  2023-02-21  21/02/2023   
11  1003990884  1003990884   6108383106  6108383106  2023-02-06  06/02/2023   
12  1003989188  1003989188   6108374159  6108374159  2023-02-02  02/02/2023   
13  1003992174  1003992174   6108391225  6108391225  2023-02-07  07/02/2023   
14  1003993769  1003993769   6108401794  6108401794  2023-02-24  24/02/2023   
15  1003985642  1003985642   6108351515  6108351515  2023-02-10  10/02/2023   
16  1003985648  1003985648   6108351514  6108351514  2023-02-10  10/02/2023   
17  1003987434  1003987434   6108363405  6108363405  2023-02-14  14/02/2023   
18  1003987374  1003987374   6108363404  6108363404  2023-02-14  14/02/2023   
19  1003988871  1003988871   6108372134  6108372134  2023-02-17  17/02/2023   
20  1003988889  1003988889   6108372133  6108372133  2023-02-17  17/02/2023   
21  1003992059  1003992059   6108390097  6108390097  2023-02-21  21/02/2023   
22  1003991986  1003991986   6108390096  6108390096  2023-02-21  21/02/2023   
23  1003987435  1003987435   6108363403  6108363403  2023-02-14  14/02/2023   
24  1003995187  1003995187   6108412853  6108412853  2023-02-28  28/02/2023   
25  1003995059  1003995059   6108382312  6108382312  2023-02-27  27/02/2023   
26  1003986443  1003986443   6108356762  6108356762  2023-02-06  06/02/2023   
27  1003986608  1003986608   6108357583  6108357583  2023-02-03  03/02/2023   
28  1003987146  1003987146   6108361440  6108361440  2023-02-17  17/02/2023   
29  1003989706  1003989706   6108377800  6108377800  2023-02-06  06/02/2023   
30  1003987107  1003987107   6108361133  6108361133  2023-02-08  08/02/2023   
31  1003987078  1003987078   6108361134  6108361134  2023-02-13  13/02/2023   
32  1003991822  1003991822   6108388145  6108388145  2023-02-20  20/02/2023   
33  1003995022  1003995022   6108410625  6108410625  2023-02-27  27/02/2023   
34  1003992060  1003992060   6108390098  6108390098  2023-02-21  21/02/2023   
35  1003993762  1003993762   6108401793  6108401793  2023-02-24  24/02/2023   
36  1003995152  1003995152   6108412852  6108412852  2023-02-28  28/02/2023   
37  1003996718  1003996718   6108425095  6108425095  2023-03-03  03/03/2023   
38  1003996322  1003996322   6108422803  6108422803  2023-02-17  17/02/2023   
39  1003994729  1003994729   6108403829  6108403829  2023-02-27  28/02/2023   
40  1003995067  1003995067   6108410628  6108410628  2023-02-27  27/02/2023   

                                         DEPOT          \
                                          self   other   
1   Vopak (Penjuru) Terminal \nPENJURU ROAD 59   VOPAK   
2   Vopak (Penjuru) Terminal \nPENJURU ROAD 59   VOPAK   
3   Vopak (Penjuru) Terminal \nPENJURU ROAD 59   VOPAK   
4   Vopak (Penjuru) Terminal \nPENJURU ROAD 59   VOPAK   
5   Vopak (Penjuru) Terminal \nPENJURU ROAD 59   VOPAK   
6   Vopak (Penjuru) Terminal \nPENJURU ROAD 59   VOPAK   
7   Vopak (Penjuru) Terminal \nPENJURU ROAD 59   VOPAK   
8   Vopak (Penjuru) Terminal \nPENJURU ROAD 59   VOPAK   
9   Vopak (Penjuru) Terminal \nPENJURU ROAD 59   VOPAK   
10  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   

In [20]:
#df_trans = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/temp_upload.csv") #Textract
trans_1 = df_trans[df_trans['DEPOT_MV2'].str.contains('VOPAK', na=False)]
#trans_1 = df_trans[~df_trans['DEPOT_MV2'].str.contains('VOPAK', na=False)]
vopak = trans_1.copy()
#vopak = vopak.iloc[:, 1:] #drop first col
vopak.reset_index(drop=True, inplace=True)

#vopak.insert(loc=0, column='Transporter Name', value='SH COGENT LOGISTICS PTE LTD')
#vopak.insert(loc=0, column='Customer Name', value='ExxonMobil Chemical Asia Pacific')

vopak = vopak.drop(columns=['Order Remarks', 'Transport No', 'DEPOT_MV2', 'LOADING_PLANT_MV3', 'DELIVERY'])
#vopak = vopak.drop(columns=['Order Remarks', 'Transport No', 'DEPOT_MV2', 'LOADING_PLANT_MV3', 'DELIVERY', 'MV2'])
#vopak.rename(columns={'Cust Ref': 'Customer Reference Number', 'Coll Date': 'Schedule Date', 'PRODUCT': 'Product name'}, inplace=True)
vopak.rename(columns={'Coll Date': 'Schedule Date', 'PRODUCT': 'Product name'}, inplace=True)

In [21]:
vopak

,Cust Ref,Schedule Date,SAP Order No,Product name
0,1003977817,2023-01-11,12728630,MCP2542 VL
1,1003994729,2023-02-27,12750473,MCP2542 VL


In [22]:
#Vopak email within 10 working days
from datetime import datetime, timedelta 
#acc = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Accepted_transports.csv")
vopak['Schedule Date'] = pd.to_datetime(vopak['Schedule Date'])
curr = datetime.now()
end = curr + timedelta(days=14)
start = curr - timedelta(days=1)

vopak_1 = vopak[vopak['Schedule Date'].between(start, end)]
lst_vopak = vopak_1['Cust Ref'].tolist()
if len(vopak_1) != 0:
    print(vopak_1)
else:
    print('No Vopak transports in next 10 business days')

send_vopak = vopak_1.copy() #make copy of vopak_1 df

     Cust Ref Schedule Date SAP Order No Product name
1  1003994729    2023-02-27     12750473   MCP2542 VL


In [24]:
def copyCertainFiles(source_folder, dest_folder, string_to_match, file_type='.pdf'):
    # Check all files in source_folder
    for filename in os.listdir(source_folder):
        # Move the file if the filename contains the string to match
        if file_type == file_type:
            if string_to_match in filename:
                shutil.move(os.path.join(source_folder, filename), dest_folder)

        # Check if the keyword and the file type both match
        elif isinstance(file_type, str):
            if string_to_match in filename and file_type in filename:
                shutil.move(os.path.join(source_folder, filename), dest_folder)

source_folder = r'C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/'
dest_folder = r'C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Send_Vopak/'
file_type = '.pdf'
import os, shutil

for cust_ref in lst_vopak:
    string_to_match = str(cust_ref)
    print(string_to_match)
    try:
        copyCertainFiles(source_folder, dest_folder, string_to_match, file_type)
    except Exception as e:
        print(e)
#close all pdf files
#https://stackoverflow.com/questions/61620485/python-if-filename-in-specified-path-contains-string-then-move-to-folder
#move pdf within 10 working days

1003994729


In [25]:
#outlook to VOPAK
from importlib import reload #send outlook multiple attachments to multiple recipients 28/11/2022 great!!
import win32com.client as win32
import pythoncom
import sys
import pdfplumber

reload(sys)
pythoncom.CoInitialize()
outlook = win32.Dispatch('outlook.application')

def sendmail(receiver, attachment, subject):
    receiver = receiver
    attachment = attachment
    sub = subject
    body = """Dear VOPAK
    <br><br>(Test email) Kindly release the booking in the system for the orders.
    <br>Customer Name: ExxonMobil Chemical Asia Pacific
    <br>Transporter Name: SH Cogent Logistics Pte Ltd<br><br>""" + vopak_1.to_html() +'<br>This message is computer generated. '+datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    mail = outlook.CreateItem(0)
    mail.To = receiver
    mail.subject = sub.encode('utf-8').decode('utf-8')
    mail.HTMLBody = body.encode('utf-8').decode('utf-8')

    for i in range(len(attachment)):
        mail.Attachments.Add(attachment[i]) # Upload multiple attachments
    mail.Send()
    
path_dir = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/" #copy Vopak pdf to Vopak folder (outlook)
dst = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/Chawan_Vopak_Email/Send_Vopak/"
for file in glob.glob(path_dir+"*.pdf"):
    #file = str(file)
    path_from = file
    pdf = pdfplumber.open(file)
    page = pdf.pages[0]
    text = page.extract_text()
    pdf.close()
    try:
        ori = regexp.search(text).group(1).split('SG-',1)[0].split()[1]
        if ori.lower() in ['VOPAK', 'vopak']:
            shutil.copy(file, dst) #if contains 'vopak', copy/paste file
            pdf.close()
    except:
        pass
    
path = r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Send_Vopak/" # Folder of attachments
addr = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Address.csv") # Email address

for j in range(len(addr.Code)):
    dirlist = []
# Address spreadsheet includes “Code” which is the recipient’s name, and “add”, the email address
    sub_code = addr.Code[j]
    #subjects = sub_code + " - transports"
    subjects = "VOPAK - transports"
    for dirpath,dirname,filename in os.walk(path):
        print(addr.Code[j])
        for i in filename:
            #if i[0:9] == 'Transport':
                dirlist.append(os.path.join(dirpath,i))
           
    sendmail(addr["add"][j], dirlist, subjects) # Send emails to multiple recipients with corresponding attachments

In [80]:
def newtrans_mail(receiver, attachment, subject):
    receiver = receiver
    attachment = attachment
    sub = subject
    body = """Dear User
    <br>(Test email) New transport orders from Transporeon.
    <br><br>""" + df_newtrans.to_html() +"<br><br>SAP Order Number<br>" + str(uploaded_trans) + '<br><br>Download latest pdf from Transporeon<br>' + df_trans.to_html() +'<br><br>This message is computer generated. '+datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    mail = outlook.CreateItem(0)
    mail.To = receiver
    mail.subject = sub.encode('utf-8').decode('utf-8')
    mail.HTMLBody = body.encode('utf-8').decode('utf-8')

    for i in range(len(attachment)):
        mail.Attachments.Add(attachment[i]) # Upload multiple attachments
    mail.Send()

##outlook to user download latest pdf Transporeon, comp.xlsx
path = r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Output/" # Folder of comp.xlsx
addr = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Address.csv") # Email address
    
for j in range(len(addr.Code)):
    dirlist = []
# Address spreadsheet includes “Code” which is the recipient’s name, and “add”, the email address
    sub_code = addr.Code[j]
    #subjects = sub_code + " - transports"
    subjects = "New - transports"
    for dirpath,dirname,filename in os.walk(path):
        print(addr.Code[j])
        for i in filename:
            #if i[0:9] == 'Transport':
                dirlist.append(os.path.join(dirpath,i))
           
    newtrans_mail(addr["add"][j], dirlist, subjects) # Send emails to multiple recipients with corresponding attachments

John
Chia Mean
Charles


In [98]:
regexp = re.compile("Destination(.*)$", re.MULTILINE) #TRANSPOREON upload to TMS works 20/1/2023!
regexp_poly = re.compile("PENJURU ROAD(.*)$", re.MULTILINE)

tms_lst = []
tms = pd.DataFrame(columns=['Delivery Type No', 'MV2', 'MV3'])

lst_cust = df_trans['LOADING_PLANT_MV3'].tolist()

#conditions=[cust=='EXXONMOBIL CHEMICAL COMPANY', cust=='MITSUI ELASTOMERS SINGAPORE PTE LTD', cust=='SUMITOMO CHEMICAL PTE LTD',
            #cust=='A.I.M. CHEMICAL INDUSTRIES PTE LTD', cust=='CLP INTERNATIONAL PTE LTD', cust=='CHAMPIONX SG SERVICE PTE LTD',
            #cust=='SINGAPORE ADHESION PLANT', cust=='BASF SOUTH EAST ASIA PTE LTD', cust=='ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD',
            #cust=='MEGACHEM MANUFACTURING PTE LTD', cust=='MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)', cust=='SINGAPORE BUTYL PLANT',
            #cust=='BERTSCHI', cust=='BAKER HUGHES', cust=='HP SINGAPORE', cust=='TOTALENERGIES', cust=='SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG']

#values=['EXX006', 'MITSUI', 'SUMITOMO CHEMICAL', 'EXX005', 'CLP001', 'CX', 'SAP', 'BASF', 'AKSS', 'MEGA', 'MSD', 'SBP', 'BERTSCHI', '627454', 'EXX002', 'TOTAL', 'SEIKA']
#values_1 = [1, 4, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1]

for cust in lst_cust:
    cust = cust.strip()
    conditions=[cust=='EXXONMOBIL CHEMICAL COMPANY', cust=='MITSUI ELASTOMERS SINGAPORE PTE LTD', cust=='SUMITOMO CHEMICAL ASIA PTE LTD',
            cust=='A.I.M. CHEMICAL INDUSTRIES PTE LTD', cust=='CLP INTERNATIONAL PTE LTD', cust=='CHAMPIONX SG SERVICE PTE LTD',
            cust=='SINGAPORE ADHESION PLANT', cust=='BASF SOUTH EAST ASIA PTE LTD', cust=='ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD',
            cust=='MEGACHEM MANUFACTURING PTE LTD', cust=='MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)', cust=='SINGAPORE BUTYL PLANT',
            cust=='BERTSCHI SINGAPORE PTE. LTD.', cust=='BAKER HUGHES', cust=='HP SINGAPORE', cust=='TOTALENERGIES MARKETING ASIA- PACIFIC MIDDLE EAST PTE LTD (LOBP)', 
            cust=='SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG', cust=='THE POLYOLEFIN - PULAU AYER MERBAU']

    values=['EXX006', 'MITSUI', 'SUMITOMO CHEMICAL', 'EXX005', 'CLP001', 'CX', 'SAP', 'BASF', 'AKSS', 'MEGA', 'MSD', 'SBP', 'BERTSCHI', '627454', 'EXX002', 'TOTAL', 'SEIKA', 'TPCPAM2']
    values_1 = [1, 4, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4]
    
    MV2 = np.select(conditions, values)
    deli_type = np.select(conditions, values_1)
    print('customer is: '+ str(cust) + ' MV2: '+ str(MV2) + ' Delivery_Type: '+str(deli_type))
    print('------------------')

    #if str(cust).strip() == 'THE POLYOLEFIN - PULAU AYER MERBAU':
            #dest = re.search(regexp_poly, text).group(0)
            #if str(dest) =='PENJURU ROAD 59 STREET 2 GATE NO 5 401' or str(dest) == 'PENJURU ROAD 59 STREET 2 GATE NO 23 401':
                #MV2 = 'TPCPAM2'
                #deli_type = 4
                #print('customer is '+ str(cust) + ' '+ str(MV2) + ' ' + str(deli_type))
            #else:
                #MV2 = 'TPCPAM1'
                #deli_type = 4
                #print('customer is '+ str(dest) + str(MV2) + ' '+ str(deli_type))
    tms_lst.extend([[str(deli_type), str(MV2), cust]])
tms = pd.DataFrame(tms_lst, columns=['Delivery_Type', 'MV2', 'LOADING_PLANT_MV3']) #list of lists to df

customer is: BASF SOUTH EAST ASIA PTE LTD MV2: BASF Delivery_Type: 1
------------------
customer is: BASF SOUTH EAST ASIA PTE LTD MV2: BASF Delivery_Type: 1
------------------
customer is: BASF SOUTH EAST ASIA PTE LTD MV2: BASF Delivery_Type: 1
------------------
customer is: BASF SOUTH EAST ASIA PTE LTD MV2: BASF Delivery_Type: 1
------------------
customer is: BERTSCHI SINGAPORE PTE. LTD. MV2: BERTSCHI Delivery_Type: 1
------------------
customer is: CHAMPIONX SG SERVICE PTE LTD MV2: CX Delivery_Type: 1
------------------
customer is: CHAMPIONX SG SERVICE PTE LTD MV2: CX Delivery_Type: 1
------------------
customer is: CHAMPIONX SG SERVICE PTE LTD MV2: CX Delivery_Type: 1
------------------
customer is: CHAMPIONX SG SERVICE PTE LTD MV2: CX Delivery_Type: 1
------------------
customer is: CHAMPIONX SG SERVICE PTE LTD MV2: CX Delivery_Type: 1
------------------
customer is: CHAMPIONX SG SERVICE PTE LTD MV2: CX Delivery_Type: 1
------------------
customer is: CLP INTERNATIONAL PTE LTD M

In [99]:
tms

,Delivery_Type,MV2,LOADING_PLANT_MV3
0,1,BASF,BASF SOUTH EAST ASIA PTE LTD
1,1,BASF,BASF SOUTH EAST ASIA PTE LTD
2,1,BASF,BASF SOUTH EAST ASIA PTE LTD
3,1,BASF,BASF SOUTH EAST ASIA PTE LTD
4,1,BERTSCHI,BERTSCHI SINGAPORE PTE. LTD.
5,1,CX,CHAMPIONX SG SERVICE PTE LTD
6,1,CX,CHAMPIONX SG SERVICE PTE LTD
7,1,CX,CHAMPIONX SG SERVICE PTE LTD
8,1,CX,CHAMPIONX SG SERVICE PTE LTD
9,1,CX,CHAMPIONX SG SERVICE PTE LTD


In [413]:
from PyPDF2 import PdfReader, PdfFileReader #TMS file development
import re

regexp = re.compile("Destination(.*)$", re.MULTILINE)
regexp_poly = re.compile("PENJURU ROAD(.*)$", re.MULTILINE)

tms_lst = []
tms = pd.DataFrame(columns=['Delivery Type No', 'MV2', 'MV3'])
path_dir = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Test/Test_2/"
for file in glob.glob(path_dir + "*.pdf"):

    pdf = pdfplumber.open(file)
    pdf_pg_count = len(pdf.pages)
    page = pdf.pages[0]
    #p = page[i]
    text = page.extract_text()
    #print(text)
    
    cust = re.search(regexp, text).group(1).split('SG-')[0] #MV3
    cust = cust.strip()
    conditions=[cust=='EXXONMOBIL', cust=='MITSUI', cust=='SUMITOMO', cust=='A.I.M. CHEMICAL', cust=='CLP INTERNATIONAL', cust=='CHAMPIONX SG', cust=='SINGAPORE', cust=='BASF SOUTH EAST', cust=='ASAHI KASEI', cust=='MEGACHEM', cust=='MSD', cust=='SINGAPORE BUTYL', cust=='BERTSCHI', cust=='BAKER HUGHES', cust=='HP SINGAPORE', cust=='TOTALENERGIES', cust=='SUMITOMO SEIKA-17']
    values=['EXX006', 'MITSUI', 'SUMITOMO CHEMICAL', 'EXX005', 'CLP001', 'CX', 'SAP', 'BASF', 'AKSS', 'MEGA', 'MSD', 'SBP', 'BERTSCHI', '627454', 'EXX002', 'TOTAL', 'SEIKA']
    values_1 = [1, 4, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1]
    
    MV2 = np.select(conditions, values)
    deli_type = np.select(conditions, values_1)
    print('customer is: '+ str(cust) + ' '+ str(MV2) + ' '+str(deli_type))
    print('------------------')
    if str(cust).strip() == 'THE POLYOLEFIN -':
            
            dest = re.search(regexp_poly, text).group(0)
            if str(dest) =='PENJURU ROAD 59 STREET 2 GATE NO 5 401' or str(dest) == 'PENJURU ROAD 59 STREET 2 GATE NO 23 401':
                MV2 = 'TPCPAM2'
                print('customer is '+ str(dest) + ' '+ str(MV2))
            else:
                MV2 = 'TPCPAM1'
                deli_type = 4
                print('customer is '+ str(dest) + str(MV2) + ' '+ str(deli_type))
            pdf.close()
    tms_lst.extend([[str(deli_type), str(MV2), cust]])
    pdf.close()
#tms_lst
tms = pd.DataFrame(tms_lst, columns=['Delivery_Type', 'MV2', 'Customer_MV3']) #list of lists to df

customer is: MITSUI MITSUI 4
------------------
customer is: MITSUI MITSUI 4
------------------
customer is: EXXONMOBIL EXX006 1
------------------
customer is: ASAHI KASEI AKSS 4
------------------
customer is: MEGACHEM MEGA 1
------------------
customer is: MEGACHEM MEGA 1
------------------
customer is: CHAMPIONX SG CX 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: THE POLYOLEFIN - 0 0
------------------
customer is PENJURU ROAD 59 STREET 2 GATE NO 23 401 TPCPAM2
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: ASAHI KASEI AKSS 4
------------------
customer is: TOTALENERGIES TOTAL 1
------------------
customer is: SUMITOMO SEIKA-17 SEIKA 1
--------------

In [103]:
#tms_upload = pd.concat([tms, df_trans], axis=1)
tms_upload = pd.merge(tms, df_trans, on='LOADING_PLANT_MV3', how='inner')
tms_upload.drop_duplicates(keep='first', inplace=True)
tms_upload.index = np.arange(1, len(tms_upload)+1)
tms_upload #template for TMS upload

,Delivery_Type,MV2,LOADING_PLANT_MV3,Cust Ref,Order Remarks,Transport No,Coll Date,DEPOT_MV2,DELIVERY,SAP Order No,PRODUCT
1,1,BASF,BASF SOUTH EAST ASIA PTE LTD,1003992060,nan,6108390098,2023-02-21,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,2023-02-21,12748370,EXXAL 8 ROP VL
2,1,BASF,BASF SOUTH EAST ASIA PTE LTD,1003993762,nan,6108401793,2023-02-24,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,2023-02-24,12748371,EXXAL 8 ROP VL
3,1,BASF,BASF SOUTH EAST ASIA PTE LTD,1003995152,nan,6108412852,2023-02-28,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,2023-02-28,12748372,EXXAL 8 ROP VL
4,1,BASF,BASF SOUTH EAST ASIA PTE LTD,1003996718,nan,6108425095,2023-03-03,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,2023-03-03,12748373,EXXAL 8 ROP VL
5,1,BERTSCHI,BERTSCHI SINGAPORE PTE. LTD.,1003995067,"TIFU7789379, TIFU7789384, TIFU7789390, PCVU263...",6108410628,2023-02-27,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,2023-02-27,12750501,SOLVESSO 100 AP VL
6,1,CX,CHAMPIONX SG SERVICE PTE LTD,1003991851,"TIFU7789379, TIFU7789384, TIFU7789390, PCVU263...",6108388149,2023-02-20,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,2023-02-20,5955541,SOLVESSO 150 AP VL
7,1,CX,CHAMPIONX SG SERVICE PTE LTD,1003993137,"TIFU7789379, TIFU7789384, TIFU7789390, PCVU263...",6108397639,2023-02-23,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,2023-02-23,5961624,SOLVESSO 150 AP VL
8,1,CX,CHAMPIONX SG SERVICE PTE LTD,1003993102,"TIFU7789379, TIFU7789384, TIFU7789390, PCVU263...",6108397638,2023-02-23,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,2023-02-23,5961625,SOLVESSO 150 AP VL
9,1,CX,CHAMPIONX SG SERVICE PTE LTD,1003992819,<b>Delivery Time</b><br>BUSINESS HOURS<br><br>...,6108395796,2023-02-14,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,2023-02-14,5969682,SOLVESSO 150 AP VL
10,1,CX,CHAMPIONX SG SERVICE PTE LTD,1003992715,"TIFU7789379, TIFU7789384, TIFU7789390, PCVU263...",6108395795,2023-02-14,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,2023-02-14,5970144,SOLVESSO 150 AP VL


In [35]:
import xlsxwriter #write df_comp

writer = pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Output/comp.xlsx", engine='xlsxwriter')
sheetComp = 'Comp_' + datetime.now().strftime("%Y%m%d_%H%M")
df_comp.to_excel(writer, sheet_name=sheetComp) #write to excel with color
worksheet1 = writer.sheets[sheetComp]
worksheet1.set_tab_color('blue')
writer.save()
writer.close()

C:\Users\john.tan\AppData\Local\Temp\ipykernel_34000\1251628687.py:8: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
c:\users\john.tan\appdata\local\programs\python\python39\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [89]:
def dataframe_difference(df1, df2, which):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(
        df2,
        indicator=True,
        how='outer'
    )
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    #diff_df.to_csv('data/diff.csv')
    return diff_df

temp_upload_1['Cust Ref'] = temp_upload_1['Cust Ref'].astype('int64')
temp_upload_1['Transport No'] = temp_upload_1['Transport No'].astype('int64')
temp_upload_1['SAP Order No'] = temp_upload_1['SAP Order No'].astype('int64')
new_upload = dataframe_difference(temp_upload_1, master, 'left_only')
new_upload = new_upload.rename(columns={'_merge': 'New_Download'})
new_upload
#temp_upload_2 = temp_upload_1.groupby('LOADING PLANT').apply(lambda x: x.sort_values('SAP Order No'))

,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No,New_Download
0,1003991851,6108388149,20/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,20/02/2023,5955541,left_only
1,1003993137,6108397639,23/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,23/02/2023,5961624,left_only
2,1003993102,6108397638,23/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,23/02/2023,5961625,left_only
3,1003991504,6108385217,03/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,MEGACHEM MANUFACTURING PTE LTD,03/02/2023,5966714,left_only
4,1003989742,6108377858,10/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,MEGACHEM MANUFACTURING PTE LTD,10/02/2023,5966715,left_only
5,1003989797,6108377859,14/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,MEGACHEM MANUFACTURING PTE LTD,14/02/2023,5966716,left_only
6,1003991839,6108388146,20/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,MEGACHEM MANUFACTURING PTE LTD,20/02/2023,5966717,left_only
7,1003992819,6108395796,14/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,14/02/2023,5969682,left_only
8,1003992715,6108395795,14/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,14/02/2023,5970144,left_only
9,1003995866,6108419506,21/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,21/02/2023,5976180,left_only


In [465]:
#optional
tp = temp_upload_1.groupby('LOADING PLANT')    
lst_tp = [tp.get_group(x) for x in tp.groups] #groupby and split df

ms = master.groupby('LOADING PLANT')
lst_ms = [ms.get_group(x) for x in ms.groups]

def repeatRows(d, n):
    return pd.concat([d]*n)

for i in range(min(len(lst_tp), len(lst_ms))):
        
    n = abs(len(lst_ms[i]) - len(lst_tp[i])) #ensure both df same len
    if n!= 0:
        if len(lst_ms[i]) > len(lst_tp[i]):
            lst_tp[i] = pd.concat([lst_tp[i], repeatRows(lst_tp[i][-1:], n)])
            #print(len(lst_tp[i]))
        else:
            lst_ms[i] = pd.concat([lst_ms[i], repeatRows(lst_ms[i][-1:], n)])
            #print(len(lst_ms[i]))
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        print(lst_ms[i].compare(lst_tp[i], keep_shape=True, keep_equal=True)) #compare side by side
        
#https://stackoverflow.com/questions/23691133/split-pandas-dataframe-based-on-groupby

     Cust Ref             Transport No              Coll Date              \
         self       other         self       other       self       other   
1  1003974560  1003974560   6108272471  6108272471  13/1/2023         NaN   
2  1003978428  1003974560   6108298482  6108272471  18/1/2023  01/11/2023   
3  1003976739  1003978428   6108288398  6108298482  19/1/2023  13/01/2023   
4  1003976739  1003979353   6108288398  6108305780  19/1/2023         NaN   

                                        DEPOT  \
                                         self   
1  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
2  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
3  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
4  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   

                                                                     PRODUCT  \
                                               other                    self   
1  T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...  EXXSOL HEXANE SP AP VL   
2 

In [474]:
#alternative comparison optional
tp6 = temp_upload_1.copy()
ms6 = master.copy()
tp6.drop(columns={'Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT'}, inplace=True)
ms6.drop(columns={'Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT'}, inplace=True)

diffs = set(ms6.iloc[:, :3].apply(tuple, 1)) \
- set(tp6.apply(tuple, 1))
lst_diffs = list(diffs)

50


In [486]:
#optional
lst_tp[11].index = np.arange(1, len(lst_tp[11])+1)
lst_ms[11].index = np.arange(1, len(lst_ms[11])+1)
lst_ms[11].compare(lst_tp[11], keep_shape=True, keep_equal=True)

Cust Ref             Transport No              Coll Date              \
         self       other         self       other       self       other   
1  1003977817  1003976424   6108295225  6108286474  11/1/2023  16/01/2023   
2  1003977817  1003977817   6108295225  6108295225  11/1/2023  18/01/2023   

                                        DEPOT  \
                                         self   
1  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
2  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   

                                                           PRODUCT  \
                                               other          self   
1  T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...  SPECTRASYN 4   
2  T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...  SPECTRASYN 4   

                                                   LOADING PLANT  \
        other                                               self   
1  MCP2542 VL  TOTALENERGIES MARKETING ASIA-PACIFIC MIDDLE EA...   
2  MCP2542 VL  TOTALENERGIES MARKETING ASIA-PACIFIC MIDDLE EA...   

                                                       DELIVERY              \
                                               other       self       other   
1  TOTALENERGIES MARKETING ASIA- PACIFIC MIDDLE E...  11/1/2023  10/01/2023   
2  TOTALENERGIES MARKETING ASIA- PACIFIC MIDDLE E...  11/1/2023  11/01/2023   

  SAP Order No            
          self     other  
1     12728630  12727043  
2     12728630  12728630

In [119]:
# (development) comp in color to excel works 17/1/2023!
temp_upload_6 = temp_upload_1.copy()
master_6 = master.copy()
tp = temp_upload_6.groupby('LOADING PLANT')   
lst_tp = [tp.get_group(x) for x in tp.groups] #groupby and split df
ms = master_6.groupby('LOADING PLANT')
lst_ms = [ms.get_group(x) for x in ms.groups]

for i in range(min(len(lst_tp), len(lst_ms))):
    n = abs(len(lst_ms[i]) - len(lst_tp[i])) #ensure both df same len
    
    if n!= 0:

        if len(lst_ms[i]) > len(lst_tp[i]):
            lst_tp[i] = pd.concat([lst_tp[i], repeatRows(lst_tp[i][-1:], n)])
        else:
            lst_ms[i] = pd.concat([lst_ms[i], repeatRows(lst_ms[i][-1:], n)])           
          
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        
        df_all = pd.concat([lst_tp[i], lst_ms[i]], axis='columns', keys=['First', 'Second'])
        df_comp = df_all.swaplevel(axis='columns')[lst_tp[i].columns[1:]]

        lst_ms[i] = lst_ms[i].dropna(subset=['Transport No','Coll Date'], how='all')
        lst_tp[i] = lst_tp[i].dropna(subset=['Transport No','Coll Date'], how='all')
            
        #lst_tp[i] = lst_tp[i].iloc[:, :-1]
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)      
           
        df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
        #df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)

        #df_comp
        with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_Test/Comp_color/comp.xlsx", mode="a", engine="openpyxl") as writer:
            df_comp.to_excel(writer, sheet_name='comp_'+str(i)+'_'+datetime.now().strftime("%Y-%m-%d"), engine='openpyxl')
               

    if n ==0:    
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        df_all = pd.concat([lst_tp[i], lst_ms[i]], axis='columns', keys=['First', 'Second'])
        df_comp = df_all.swaplevel(axis='columns')[lst_tp[i].columns[1:]]

        lst_ms[i] = lst_ms[i].dropna(subset=['Transport No','Coll Date'], how='all')
        lst_tp[i] = lst_tp[i].dropna(subset=['Transport No','Coll Date'], how='all')
            
        #lst_tp[i] = lst_tp[i].iloc[:, :-1]
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
                
        df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
        #df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)

        with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Output/comp.xlsx", mode="a", engine="openpyxl") as writer:
            df_comp.to_excel(writer, sheet_name='comp_'+str(i)+'_'+datetime.now().strftime("%Y-%m-%d"), engine='openpyxl')

C:\Users\john.tan\AppData\Local\Temp\ipykernel_29352\571956376.py:53: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
C:\Users\john.tan\AppData\Local\Temp\ipykernel_29352\571956376.py:53: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
C:\Users\john.tan\AppData\Local\Temp\ipykernel_29352\571956376.py:32: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
C:\Users\john.tan\AppData\Local\Temp\ipykernel_29352\571956376.py:32: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keywor

In [90]:
#compare Upcoming with master. Works 02/02/2023!
upc = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Upcoming.csv", usecols=['Order No'])
upc.rename(columns={'Order No': 'SAP Order No'}, inplace=True) #'Deliv Date': 'DELIVERY'
#upc['DELIVERY'] = pd.to_datetime(upc['DELIVERY'], format='%Y%m%d').dt.strftime('%d/%m/%Y')
upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
upc.index = np.arange(1, len(upc)+1)
lst_upc = upc['SAP Order No'].tolist()

new_upc = [] #convert list of int to str
for x in lst_upc:
    str_x = "% s" % x
    new_upc.append(str_x)

master_upc = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/EM master file.csv", usecols=['EM SO# Ref'])
master_upc.rename(columns={'EM SO# Ref': 'SAP Order No'}, inplace=True)
master_upc = master_upc[master_upc['SAP Order No'].isin(new_upc)]
master_upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
master_upc.index = np.arange(1, len(master_upc)+1)

sap_master_upc = master_upc['SAP Order No'].tolist()

master_upc_1 = master_upc.copy()
master_upc_1['SAP Order No'] = master_upc_1['SAP Order No'].astype(np.int64) #col to int
upOnly = dataframe_difference(upc, master_upc_1, 'left_only')

upOnly.rename(columns={"_merge": "New_Transport"}, inplace=True)
upOnly['New_Transport'] = upOnly['New_Transport'].replace('left_only', 'new upcoming')
upOnly.index = np.arange(1, len(upOnly)+1)
print('\033[1m'+ '\033[34m'+ magenta(('Transports from Upcoming Shipments Report not in MasterFile'), ['bold', 'underlined'])) #'\033[94m'
upOnly

Transports from Upcoming Shipments Report not in MasterFile


,SAP Order No,New_Transport
1,12720815,new upcoming
2,12726060,new upcoming
3,12732175,new upcoming
4,12732176,new upcoming
5,12732177,new upcoming
6,12732178,new upcoming
7,4452380265,new upcoming
8,4452380265,new upcoming
9,4452380265,new upcoming
10,4452380265,new upcoming


In [91]:
upc['SAP Order No'] = upc['SAP Order No'].astype(np.int64)
master_upc_1['SAP Order No'] = master_upc_1['SAP Order No'].astype(np.int64)
arr_upc = np.array(lst_upc)
arr_master_upc = np.array(sap_master_upc)
dif1 = np.setdiff1d(arr_upc, arr_master_upc)
dif2 = np.setdiff1d(arr_master_upc, arr_upc)

lst_upOnly = np.concatenate((dif1, dif2))
print('\033[1m'+ '\033[34m'+ 'SAP Numbers from Upcoming Shipments Report not in MasterFile')
lst_upOnly

SAP Numbers from Upcoming Shipments Report not in MasterFile


array(['12720815', '12726060', '12732175', '12732176', '12732177',
       '12732178', '4452380265'], dtype='<U21')

In [92]:
upc_only = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Upcoming.csv")
upc_only['Order No'] = upc_only['Order No'].astype('str')
upc_only = upc_only[upc_only['Order No'].isin(lst_upOnly)]
print('\033[1m'+ '\033[34m'+ 'Upcoming Shipments Report not in MasterFile')
upc_only

Upcoming Shipments Report not in MasterFile


,Ship No,Carrier No,Carrier Name,Deliv No,Order No,Item,Material No,Batch No,Del It,Qty/Batch,...,Region,Country,Deliv Date,Ship Load Date,Delivery Qty,Delivery Qty (KG),UOM,Plant,TM FO No,General Text
8,1003980889,SE826738,SH COGENT LOGISTICS PTE LTD,879597957,12720815,10,5221430,NaN,10,15000.0,...,SG,SG,20230127,20230127,15000.0,15000,KG,SG03,6108314617,"Header Prep: N/A, Item Text: TIFU 1526380, TIF..."
17,1003978481,SE826738,SH COGENT LOGISTICS PTE LTD,879583239,4452380265,10,5054536,NaN,10,17.5,...,SG,SG,20230113,20230113,17.5,17500,MT,SGV3,6108298922,"Header Prep: N/A, Item Text: N/A"
23,1003978502,SE826738,SH COGENT LOGISTICS PTE LTD,879583240,4452380265,10,5054536,NaN,10,17.5,...,SG,SG,20230116,20230116,17.5,17500,MT,SGV3,6108298923,"Header Prep: N/A, Item Text: N/A"
26,1003978503,SE826738,SH COGENT LOGISTICS PTE LTD,879583241,4452380265,10,5054536,NaN,10,17.5,...,SG,SG,20230117,20230117,17.5,17500,MT,SGV3,6108298924,"Header Prep: N/A, Item Text: N/A"
28,1003978504,SE826738,SH COGENT LOGISTICS PTE LTD,879583242,4452380265,10,5054536,NaN,10,17.5,...,SG,SG,20230118,20230118,17.5,17500,MT,SGV3,6108298925,"Header Prep: N/A, Item Text: N/A"
30,1003978505,SE826738,SH COGENT LOGISTICS PTE LTD,879583243,4452380265,10,5054536,NaN,10,17.5,...,SG,SG,20230119,20230119,17.5,17500,MT,SGV3,6108298926,"Header Prep: N/A, Item Text: N/A"
32,1003978506,SE826738,SH COGENT LOGISTICS PTE LTD,879583244,4452380265,10,5054536,NaN,10,17.5,...,SG,SG,20230120,20230120,17.5,17500,MT,SGV3,6108298927,"Header Prep: N/A, Item Text: N/A"
33,1003979353,SE826738,SH COGENT LOGISTICS PTE LTD,879590407,12726060,10,5054643,NaN,10,15000.0,...,SG,SG,20230125,20230125,15000.0,15000,KG,SGV3,6108305780,"Header Prep: N/A, Item Text: TIFU 1526380, TIF..."
38,1003979615,SE826738,SH COGENT LOGISTICS PTE LTD,879591471,12732175,10,5073027,NaN,10,17000.0,...,NaN,SG,20230113,20230113,17000.0,17000,KG,SGV6,6108308007,"Header Prep: N/A, Item Text: 44523211612 SNTU ..."
39,1003979687,SE826738,SH COGENT LOGISTICS PTE LTD,879591472,12732176,10,5073027,NaN,10,17000.0,...,NaN,SG,20230114,20230114,17000.0,17000,KG,SGV6,6108308008,"Header Prep: N/A, Item Text: 44523211613 TPMU ..."


In [35]:
#compare Upcoming with master (development)
upc = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Upcoming.csv", usecols=['Order No', 'Deliv Date', 'Customer Name'])
upc.rename(columns={'Order No': 'SAP Order No', 'Customer Name': 'LOADING', 'Deliv Date': 'DELIVERY'}, inplace=True)
upc['DELIVERY'] = pd.to_datetime(upc['DELIVERY'], format='%Y%m%d').dt.strftime('%d/%m/%Y')
upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
lst_upc = upc['SAP Order No'].tolist()

new_upc = [] #convert list of int to str
for x in lst_upc:
    str_x = "% s" % x
    new_upc.append(str_x)

master_upc = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/EM master file.csv", usecols=['EM SO# Ref', 'DELIVERY', 'LOADING PLANT'])
master_upc.rename(columns={'EM SO# Ref': 'SAP Order No', 'LOADING PLANT': 'LOADING'}, inplace=True)
master_upc = master_upc[master_upc['SAP Order No'].isin(new_upc)]
master_upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')

sap_master_upc = master_upc['SAP Order No'].tolist()
#print(sap_master_upc) #list of str
#print(lst_upc) #list of int

upc['SAP Order No'] = upc['SAP Order No'].astype(str) #convert col to str
upc_notin_master = upc[~upc['SAP Order No'].isin(sap_master_upc)] #select Upcoming not in Master
upc_notin_master.index = np.arange(1, len(upc_notin_master)+1)
#print('Upcoming transports not in Master file')
#print(upc_notin_master)

idx1 = set(upc.set_index(['SAP Order No', 'LOADING', 'DELIVERY']).index)
idx2 = set(upc_notin_master.set_index(['SAP Order No', 'LOADING', 'DELIVERY']).index)
upc_in_master = pd.DataFrame(list(idx1 - idx2), columns=upc.columns) #select Upcoming in Master

upc_in_master.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
upc_in_master.index = np.arange(1, len(upc_in_master)+1)
print('Upcoming transports in Master file')
print(upc_in_master)

master_upc.index = np.arange(1, len(master_upc)+1)
print('Master file')
print(master_upc)

#diffs = set(master_upc.iloc[:, :3].apply(tuple, 1)) \
                       #-  set(upc.apply(tuple, 1))
#print('Difference in master ')
#print(diffs)
diffs_2 = set(upc.iloc[:, :3].apply(tuple, 1)) \
                       -  set(master_upc.apply(tuple, 1))
#print('New entry in Upcoming transports')
DF_diffs_2 = pd.DataFrame(data=list(diffs_2), columns=['SAP Order No', 'LOADING', 'DELIVERY'])
#DF_diffs_2
#https://stackoverflow.com/questions/46635439/compare-rows-of-two-data-frames-with-unequal-lengths
#https://stackoverflow.com/questions/37313691/how-to-remove-a-pandas-dataframe-from-another-dataframe

Upcoming transports in Master file
Empty DataFrame
Columns: [SAP Order No, LOADING, DELIVERY]
Index: []
Master file
Empty DataFrame
Columns: [SAP Order No, LOADING, DELIVERY]
Index: []


In [110]:
from importlib import reload #send outlook multiple recipients 18/1/2023 great!!
import win32com.client as win32
import pythoncom

reload(sys)
pythoncom.CoInitialize()
outlook = win32.Dispatch('outlook.application')

def upctrans_mail(receiver, attachment, subject):
    receiver = receiver
    #attachment = attachment
    sub = subject
    body = """Dear User
    <br>(Test email) EM MasterFile.
    <br><br>""" + master_upc.to_html() +"<br><br>Upcoming Shipments not in MasterFile<br>" + str(lst_upOnly)+'<br>'+ upc_only.to_html()+'<br><br>This message is computer generated. '+datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    mail = outlook.CreateItem(0)
    mail.To = receiver
    mail.subject = sub.encode('utf-8').decode('utf-8')
    mail.HTMLBody = body.encode('utf-8').decode('utf-8')

    #for i in range(len(attachment)):
        #mail.Attachments.Add(attachment[i]) # Upload multiple attachments
    mail.Send()

##outlook to user download latest pdf Transporeon, comp.xlsx
path = r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Test/Upcoming/" # Folder of comp.xlsx
addr = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Address.csv") # Email address
    
for j in range(len(addr.Code)):
    dirlist = []
# Address spreadsheet includes “Code” which is the recipient’s name, and “add”, the email address
    sub_code = addr.Code[j]
    #subjects = sub_code + " - transports"
    subjects = "Upcoming - transports"
    for dirpath,dirname,filename in os.walk(path):
        print(addr.Code[j])
        for i in filename:
            #if i[0:9] == 'Transport':
                dirlist.append(os.path.join(dirpath,i))
           
    upctrans_mail(addr["add"][j], dirlist, subjects) # Send emails to multiple recipients with corresponding attachments

John


In [93]:
temp_upload_diff = temp_upload_1[['SAP Order No', 'LOADING PLANT', 'DELIVERY']]
temp_upload_diff.rename(columns={'LOADING PLANT': 'LOADING'}, inplace=True)
diffs_3 = set(temp_upload_diff.iloc[:, :3].apply(tuple, 1)) \
                       -  set(master.apply(tuple, 1))
print('New entry in Uploaded transports')
DF_diffs_3 = pd.DataFrame(data=list(diffs_3), columns=['SAP Order No', 'LOADING', 'DELIVERY'])
DF_diffs_3 #Upcoming not in master

New entry in Uploaded transports


C:\Users\john.tan\AppData\Local\Temp\ipykernel_1056\2230247851.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_upload_diff.rename(columns={'LOADING PLANT': 'LOADING'}, inplace=True)


,SAP Order No,LOADING,DELIVERY
0,12759254,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH),06/03/2023
1,5966716,MEGACHEM MANUFACTURING PTE LTD,14/02/2023
2,5966715,MEGACHEM MANUFACTURING PTE LTD,10/02/2023
3,12737576,THE POLYOLEFIN - PULAU AYER MERBAU,07/02/2023
4,12739935,THE POLYOLEFIN - PULAU AYER MERBAU,28/02/2023
5,5969682,CHAMPIONX SG SERVICE PTE LTD,14/02/2023
6,5966714,MEGACHEM MANUFACTURING PTE LTD,03/02/2023
7,12737636,MITSUI ELASTOMERS SINGAPORE PTE LTD,21/02/2023
8,4911877915,EXXONMOBIL CHEMICAL COMPANY,02/02/2023
9,12758287,MITSUI ELASTOMERS SINGAPORE PTE LTD,03/03/2023


In [73]:
#df3 = df2.reindex_like(df1)
#df1[(df3['a'].isin(df1['a'])) & (df1['b'] <= df3['b'])]
comp_6 = temp_upload_1.reindex_like(master)
master[(comp_6['Cust Ref'].isin(master['Cust Ref'])) & (master['PRODUCT'] <= comp_6['PRODUCT'])]
#master[np.logical_not(comp_6['Cust Ref'].isin(master['Cust Ref'])) & (master['DEPOT'] <= comp_6['DEPOT'])]

,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No


In [37]:
#writer1 = pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_Test/temp_upload.xlsx", engine='xlsxwriter')
#temp_upload_1.to_excel(writer1, sheet_name='temp_upload') #write to excel with color
#worksheet1 = writer1.sheets['temp_upload']
#worksheet1.set_tab_color('blue')
#writer1.save()
#writer1.close()

with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Output/NewTransports_ChawanVopak.xlsx", mode='a', if_sheet_exists='replace', engine='openpyxl') as writer3:
    sheetName = 'NewTransports_' + datetime.now().strftime("%Y%m%d %H%M")
    temp_upload_1.to_excel(writer3, sheet_name=sheetName, engine='openpyxl')
    worksheet3 = writer3.sheets[sheetName]
    writer3.save()

with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Output/NewTransports_50.xlsx", mode='a', if_sheet_exists='replace', engine='openpyxl') as writer4:
    sheetName = 'Transporeon50_' + datetime.now().strftime("%Y%m%d %H%M")
    df_trans.to_excel(writer4, sheet_name=sheetName, engine='openpyxl')
    worksheet4 = writer4.sheets[sheetName]
    writer4.save()
  
with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/AI Prime Mover/NewTransports.xlsx", mode='a', if_sheet_exists='replace', engine='openpyxl') as writer5:
    sheetName = 'NewTransports_' + datetime.now().strftime("%Y%m%d %H%M")
    df_newtrans.to_excel(writer5, sheet_name=sheetName, engine='openpyxl')
    worksheet5 = writer5.sheets[sheetName]
    writer5.save()
print('\033[1m'+ '\033[32m' + 'Process completed')

C:\Users\john.tan\AppData\Local\Temp\ipykernel_34000\2521141350.py:12: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer3.save()
C:\Users\john.tan\AppData\Local\Temp\ipykernel_34000\2521141350.py:18: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer4.save()


Process completed


C:\Users\john.tan\AppData\Local\Temp\ipykernel_34000\2521141350.py:24: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer5.save()


In [72]:
#development kv_pairs to df
for kv_pair in result.key_value_pairs:
            if kv_pair.key and kv_pair.value:
                lst_trans.extend([[kv_pair.key.content, kv_pair.value.content]])
                #print("Key '{}': Value: '{}'".format(kv_pair.key.content, kv_pair.value.content))
            else:
                print("Key '{}': Value:".format(kv_pair.key.content))
                print('----------------------')
  
            items = result.key_value_pairs    
            #i += 1
#lst_trans

#field_lst = ["Cust Ref", "Order Remarks", "Transport No", "Coll Date", "DEPOT_MV2", "LOADING_PLANT_MV3", "DELIVERY", "SAP Order No", "MV2"]
field_lst = ['Shipping Number (BOL)', 'Comment', 'Transport No.', 'Deadline', 'Origin', 'Destination',  'Deadline', 'SAP Order No.', 'Article description']#MV2
field_lst2 = ['Comment', 'Transport No.', 'Deadline', 'Origin', 'Destination',  'Deadline', 'SAP Order No.', 'Article description']#MV2
qq = pd.DataFrame(lst_trans, columns=['SN', 'Va'])
qqq = qq.set_index('SN').T.reset_index(drop=True)
try:
    q = qqq[field_lst]
except:
    pass
    q = qqq[field_lst2]
q = q.loc[:, ~q.columns.duplicated()].copy() #remove dup cols
q.rename(columns={'Comment': 'Order Remarks', 'Transport No.': 'Transport No', 'Deadline': 'Coll Date', 'Origin': 'DEPOT_MV2', 'Destination': 'LOADING_PLANT_MV3', 'SAP Order No.': 'SAP Order No', 'Article description': 'PRODUCT'}, inplace=True)

q['TIME SLOT'] = [x.split(' ')[1] for x in q['Coll Date']]
q['Coll Date'] = [x.split(' ')[0].replace(' ', '') for x in q['Coll Date']]
q['DELIVERY'] = q['Coll Date']
#q.drop(columns=q.columns[0], axis=1, inplace=True)

col_reord = ['Order Remarks', 'Transport No', 'Coll Date', 'DEPOT_MV2', 'LOADING_PLANT_MV3', 'DELIVERY', 'SAP Order No', 'PRODUCT', 'TIME SLOT']
new_cols = col_reord + (q.columns.drop(col_reord).tolist())
q = q[new_cols]
q

Key 'Origin': Value:
----------------------
Key 'Bill-To partner code': Value:
----------------------


SN,Order Remarks,Transport No,Coll Date,DEPOT_MV2,LOADING_PLANT_MV3,DELIVERY,SAP Order No,PRODUCT,TIME SLOT,Shipping Number (BOL)
0,<b>Delivery Time</b><br>BUSINESS HOURS<br><br>...,6108412853,2023-02-13,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,THE POLYOLEFIN\nPULAU AYER MERBAU\nSG-628285 S...,2023-02-13,12739935,EXXSOL HEXANE SP AP VL,11:58,1003995187


In [71]:
for kv_pair in result.key_value_pairs:
            if kv_pair.key and kv_pair.value:
                lst_trans.extend([[kv_pair.key.content, kv_pair.value.content]])
                #print("Key '{}': Value: '{}'".format(kv_pair.key.content, kv_pair.value.content))
lst_trans          

[['Transport No.', '6108412853'],
 ['Printed:', '2023-02-13 09:43'],
 ['Delivery numbers:', '6108412853-0000000010'],
 ['Last modified:', '2023-02-11 18:35'],
 ['Transporeon-ID', '788129831'],
 ['Deadline', '2023-02-13 11:58'],
 ['Weight', '15,000.00 kg'],
 ['Carrier', 'SH Cogent Logistics Pte Ltd 1 Buroh Crescent #6M-01'],
 ['Volume', '22.85 cbm'],
 ['Vehicle (Requirement)', 'AP - Liquid Road Truck 24T'],
 ['Shipper', 'SG-627545 Singapore'],
 ['Company Scheduling unit', 'Standard'],
 ['Scheduler', 'TP XP Service Account'],
 ['Company name', 'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal'],
 ['Destination',
  'THE POLYOLEFIN\nPULAU AYER MERBAU\nSG-628285 SINGAPORE (DDP)\n2023-02-28 08:00\n17:00\n15,000 kg\n22,845.00'],
 ['Transport comment:',
  'General Text\nTPC1 - TIFU 1526380, TIFU 1526396, TIFU 1526400'],
 ['Bill-To partner name', 'ExxonMobil Chemical Asia Pacific'],
 ['Bill-To partner address',
  '(A Division of ExxonMobil Asia Pacific Pte Ltd)'],
 ['Bill-To partner city', '